# Review Rec Bot

# Set up

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from loguru import logger
import json

import sys
sys.path.insert(0, '..')

In [3]:
import os

from tqdm.notebook import tqdm
import pandas as pd
from llama_index.core.response.notebook_utils import display_source_node

In [4]:
import nest_asyncio

nest_asyncio.apply()

In [5]:
from dotenv import load_dotenv

load_dotenv()

True

# Arguments

In [6]:
from src.run.args import RunInputArgs

ARGS = RunInputArgs(
    EXPERIMENT_NAME="Review Rec Bot - Yelp Review Rec Bot",
    RUN_NAME="026_rez_tool",
    RUN_DESCRIPTION="""
# Objective
Try adding the Reservation Availability Tool to Agent Toolkit

# Implementation
- Need to reindex to account for the added metadata
- Add field `business_id` available to LLM so that they can use to make call to the functions

## Changelog
""",
    TESTING=False,
    LOG_TO_MLFLOW=True,
    OBSERVABILITY=True,
    RECREATE_INDEX=True,
    RECREATE_RETRIEVAL_EVAL_DATASET=True,
    RECREATE_RESPONSE_EVAL_DATASET=True,
    DEBUG=False,
)

ARGS

{
  "EXPERIMENT_NAME": "Review Rec Bot - Yelp Review Rec Bot",
  "RUN_NAME": "026_rez_tool",
  "RUN_DESCRIPTION": "\n# Objective\nTry adding the Reservation Availability Tool to Agent Toolkit\n\n# Implementation\n- Need to reindex to account for the added metadata\n- Add field `business_id` available to LLM so that they can use to make call to the functions\n\n## Changelog\n",
  "TESTING": false,
  "DEBUG": false,
  "OBSERVABILITY": true,
  "LOG_TO_MLFLOW": true,
  "RECREATE_RETRIEVAL_EVAL_DATASET": true,
  "RECREATE_RESPONSE_EVAL_DATASET": true,
  "RECREATE_INDEX": true
}

# Load config

In [7]:
from src.run.cfg import RunConfig

In [8]:
cfg = RunConfig()
cfg.init(ARGS)

2024-08-13 21:09:29.511 | INFO     | src.run.cfg:init:148 - Starting Observability server with Phoenix...
INFO:phoenix.config:📋 Ensuring phoenix working directory: /home/dvquys/.phoenix
2024-08-13 21:09:34.232 | INFO     | src.run.cfg:init:166 - Setting up MLflow experiment Review Rec Bot - Yelp Review Rec Bot - run 026_rez_tool...


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


2024-08-13 21:09:34.509 | INFO     | src.run.cfg:init:175 - Notebook-generated artifacts are persisted at data/026_rez_tool
2024-08-13 21:09:34.510 | INFO     | src.run.cfg:init:181 - ARGS.RECREATE_INDEX=True -> Overwriting db_collection and storage_context_persist_dp...


In [9]:
cfg

/home/dvquys/frostmourne/study/vietai-genai03/final-project/review-rec-bot/.venv/lib/python3.11/site-packages/pydantic/main.py:364: UserWarning: Pydantic serializer warnings:
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


{
  "args": {
    "EXPERIMENT_NAME": "Review Rec Bot - Yelp Review Rec Bot",
    "RUN_NAME": "026_rez_tool",
    "RUN_DESCRIPTION": "\n# Objective\nTry adding the Reservation Availability Tool to Agent Toolkit\n\n# Implementation\n- Need to reindex to account for the added metadata\n- Add field `business_id` available to LLM so that they can use to make call to the functions\n\n## Changelog\n",
    "TESTING": false,
    "DEBUG": false,
    "OBSERVABILITY": true,
    "LOG_TO_MLFLOW": true,
    "RECREATE_RETRIEVAL_EVAL_DATASET": true,
    "RECREATE_RESPONSE_EVAL_DATASET": true,
    "RECREATE_INDEX": true
  },
  "app_name": "review_rec_bot",
  "storage_context_persist_dp": "data/026_rez_tool/storage_context",
  "db_collection": "review_rec_bot__026_rez_tool__huggingface____data_finetune_embedding_finetuned_model",
  "notebook_cache_dp": "data/026_rez_tool",
  "data_fp": "../data/yelp_dataset/sample/sample_100_biz/denom_review.parquet",
  "llm_cfg": {
    "llm_provider": "openai",
    "llm

## Set up logger to collect additional info

In [10]:
collect_fp = f"{cfg.notebook_cache_dp}/collect.log"
logger.add(collect_fp, filter=lambda record: "[COLLECT]" in record['message'], mode='w')

1

# Load input data

In [11]:
data = pd.read_parquet(cfg.data_fp)
data = data.assign(
    biz_categories=lambda df: df['biz_categories'].str.split(', '),
    date=lambda df: df['date'].dt.strftime('%Y-%m-%dT%H:%M:%S')
)
logger.info(f"[COLLECT] {len(data)=}")
data.iloc[0]

2024-08-13 21:09:34.675 | INFO     | __main__:<module>:6 - [COLLECT] len(data)=5240


review_id                                      L0jv8c2FbpWSlfNC6bbUEA
user_id                                        bFPdtzu11Oi0f92EAcjqmg
business_id                                    IDtLPgUrqorrpqSLdfMhZQ
review_stars                                                        5
useful                                                              0
funny                                                               0
cool                                                                0
text                What a great addition to the Funk Zone!  Grab ...
date                                              2016-10-13T22:50:47
biz_name                                         Helena Avenue Bakery
biz_address                                     131 Anacapa St, Ste C
biz_city                                                Santa Barbara
biz_state                                                          CA
biz_postal_code                                                 93101
biz_latitude        

In [12]:
input_data = data
if ARGS.TESTING:
    input_data = data[:20]
logger.info(f"[COLLECT] {len(input_data)=}")

2024-08-13 21:09:34.708 | INFO     | __main__:<module>:4 - [COLLECT] len(input_data)=5240


In [13]:
input_data.columns

Index(['review_id', 'user_id', 'business_id', 'review_stars', 'useful',
       'funny', 'cool', 'text', 'date', 'biz_name', 'biz_address', 'biz_city',
       'biz_state', 'biz_postal_code', 'biz_latitude', 'biz_longitude',
       'biz_stars', 'biz_review_count', 'biz_is_open', 'biz_attributes',
       'biz_categories', 'biz_hours'],
      dtype='object')

# Prepare documents

In [14]:
from llama_index.core import Document

documents = []
embedding_visible_metadata = ["review_stars", "biz_name", "biz_address", "biz_city", "biz_state", "biz_categories", "business_id"]
excluded_embed_metadata_keys = [k for k in input_data.columns if k not in embedding_visible_metadata]

for i, row in tqdm(input_data.iterrows(), total=len(input_data)):
    record = row.to_dict()
    text = record['text']
    metadata = {k: v for k, v in record.items() if k not in ('text')}
    
    doc = Document(
        text=text,
        metadata=metadata,
        excluded_embed_metadata_keys=excluded_embed_metadata_keys,
        excluded_llm_metadata_keys=excluded_embed_metadata_keys
    )
    documents.append(doc)

logger.info(f"[COLLECT] {len(documents)=}")

  0%|          | 0/5240 [00:00<?, ?it/s]

2024-08-13 21:09:35.249 | INFO     | __main__:<module>:20 - [COLLECT] len(documents)=5240


#### Check document embedding text

In [15]:
from llama_index.core.schema import MetadataMode

In [16]:
document = documents[0]
print(document.get_content(metadata_mode=MetadataMode.EMBED))

business_id: IDtLPgUrqorrpqSLdfMhZQ
review_stars: 5
biz_name: Helena Avenue Bakery
biz_address: 131 Anacapa St, Ste C
biz_city: Santa Barbara
biz_state: CA
biz_categories: ['Food', 'Restaurants', 'Salad', 'Coffee & Tea', 'Breakfast & Brunch', 'Sandwiches', 'Bakeries']

What a great addition to the Funk Zone!  Grab a bite, grab some tastings, life is good. Right next door to the Santa Barbara Wine Collective, in fact it actually shares the same tables.  We had a fabulous savory croissant.


# Set up LLM

In [17]:
llm, embed_model = cfg.setup_llm()

WARNI [sentence_transformers.SentenceTransformer] You try to use a model that was created with version 2.7.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





In [18]:
print(cfg.llm_cfg.model_dump_json(indent=2))

{
  "llm_provider": "openai",
  "llm_model_name": "gpt-4o-mini",
  "embedding_provider": "huggingface",
  "embedding_model_name": "./data/finetune_embedding/finetuned_model",
  "embedding_model_dim": 768,
  "ollama__host": "192.168.100.14",
  "ollama__port": 11434
}


In [19]:
from llama_index.core import Settings
Settings.embed_model = embed_model
Settings.llm = llm

# Vector Store

In [20]:
import qdrant_client
from llama_index.core import VectorStoreIndex
from llama_index.core import StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore

from src.run.orchestrator import RunOrchestrator

In [21]:
qdrantdb = qdrant_client.QdrantClient(
    host="localhost",
    port=6333
)
aqdrantdb = qdrant_client.AsyncQdrantClient(
    host="localhost",
    port=6333
)

RunOrchestrator.setup_db(cfg, qdrantdb)

db_collection = qdrantdb.get_collection(cfg.db_collection)
vector_store = QdrantVectorStore(
    client=qdrantdb,
    collection_name=cfg.db_collection,
    aclient=aqdrantdb,
    enable_hybrid=False,
    prefer_grpc=True
)

2024-08-13 21:09:46.047 | INFO     | src.run.orchestrator:setup_db:29 - Creating new Qdrant collection review_rec_bot__026_rez_tool__huggingface____data_finetune_embedding_finetuned_model...
WARNI [llama_index.vector_stores.qdrant.base] Both client and aclient are provided. If using `:memory:` mode, the data between clients is not synced.


# Index Embeddings

In [22]:
import time
import pickle
from multiprocessing import set_start_method
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core.node_parser import SemanticSplitterNodeParser
# from llama_index.core.node_parser import SentenceSplitter

In [23]:
chunker = SemanticSplitterNodeParser
chunker_cfg = {
    "buffer_size": 1,
    "breakpoint_percentile_threshold": 95,
    "embed_model": embed_model
}

# chunker = SentenceSplitter
# chunker_cfg = {
#     "chunk_size": 512,
#     "chunk_overlap": 10
# }

In [24]:
t0 = time.perf_counter()
# TODO: TO understand the differences between points_count and indexed_vector_counts.
# Here indexed_vector_counts = 0
db_collection_count = db_collection.points_count

if db_collection_count > 0 and ARGS.RECREATE_INDEX == False:
    logger.info(f"Loading Storage Context from {cfg.storage_context_persist_dp}...")
    docstore = SimpleDocumentStore.from_persist_dir(persist_dir=cfg.storage_context_persist_dp)
    storage_context = StorageContext.from_defaults(
        docstore=docstore,
        vector_store=vector_store
    )
    nodes = list(docstore.docs.values())
else:
    logger.info(f"Creating new DB index...")
    from llama_index.core.extractors import TitleExtractor
    from llama_index.core.ingestion import IngestionPipeline, IngestionCache

    # create the pipeline with transformations
    pipeline = IngestionPipeline(
        transformations=[
            chunker(**chunker_cfg),
            embed_model,
        ],
        vector_store = vector_store
    )

    num_workers = None
    # TODO: I can get the preprocessing completed much quicker if I push the initial steps through a multiprocessing loop and then separately create the embeddings using the built-in batching already provided in the SentenceTransformer encode method.
    # Ref: https://github.com/run-llama/llama_index/issues/10104#issuecomment-1899401584
    # Currently setting num_workers leads to code simple hang
    # Ref: https://github.com/run-llama/llama_index/issues/10104
    # num_workers = os.cpu_count() - 1
    # os.environ['TOKENIZERS_PARALLELISM'] = 'true'
    # set_start_method("spawn", force=True)  # it hangs without this line
    logger.info(f"Running Ingestion Pipeline with {num_workers=}...")
    nodes = await pipeline.arun(documents=documents, num_workers=num_workers, show_progress=True)

    # Define Docstore as an abstraction on top of nodes to easily manage (e.g. get node by id)
    docstore = SimpleDocumentStore()
    await docstore.async_add_documents(nodes)
    storage_context = StorageContext.from_defaults(
        docstore=docstore,
        vector_store=vector_store
    )
    logger.info(f"Persisting Storage Context to {cfg.storage_context_persist_dp}...")
    storage_context.persist(cfg.storage_context_persist_dp)

t1 = time.perf_counter()

2024-08-13 21:09:54.055 | INFO     | __main__:<module>:15 - Creating new DB index...
2024-08-13 21:09:54.073 | INFO     | __main__:<module>:36 - Running Ingestion Pipeline with num_workers=None...


Parsing nodes:   0%|          | 0/5240 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/4 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/4 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/9 [00:00<?, ?it/s]

...

Generating embeddings:   0%|          | 0/7 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/18 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/28 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/10 [00:00<?, ?it/s]

Generating embeddings: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2555/2555 [03:55<00:00, 10.87it/s]
2024-08-13 21:23:25.141 | INFO     | __main__:<module>:46 - Persisting Storage Context to data/026_rez_tool/storage_context...


In [25]:
logger.info(f"Indexing {len(documents)} documents into VectorStoreIndex took {t1 - t0:,.0f}s")
logger.info(f"[COLLECT] {len(nodes)=}")

2024-08-13 21:23:38.053 | INFO     | __main__:<module>:1 - Indexing 5240 documents into VectorStoreIndex took 815s
2024-08-13 21:23:38.054 | INFO     | __main__:<module>:2 - [COLLECT] len(nodes)=10219


# Analyze Chunks

In [26]:
for i, node in enumerate(nodes[:5]):
    print(f"\n\n==========Node {i+1}============")
    print(node.metadata)
    print(node.get_text())



==========Node 1============
{'review_id': 'L0jv8c2FbpWSlfNC6bbUEA', 'user_id': 'bFPdtzu11Oi0f92EAcjqmg', 'business_id': 'IDtLPgUrqorrpqSLdfMhZQ', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2016-10-13T22:50:47', 'biz_name': 'Helena Avenue Bakery', 'biz_address': '131 Anacapa St, Ste C', 'biz_city': 'Santa Barbara', 'biz_state': 'CA', 'biz_postal_code': '93101', 'biz_latitude': 34.4144445, 'biz_longitude': -119.6906718, 'biz_stars': 4.0, 'biz_review_count': 389, 'biz_is_open': 1, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'none'", 'Ambience': "{'touristy': False, 'hipster': True, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': True, 'upscale': False, 'classy': False, 'casual': True}", 'BYOB': None, 'BestNights': None, 'BikeParking': 'True', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}", 'ByAppointmentOnly':

# Construct Retriever

In [27]:
from llama_index.core.retrievers import VectorIndexRetriever

In [28]:
print(cfg.retrieval_cfg.model_dump_json(indent=2))

{
  "retrieval_top_k": 50,
  "retrieval_dense_top_k": 50,
  "retrieval_sparse_top_k": 50,
  "retrieval_similarity_cutoff": null,
  "rerank_top_k": 10,
  "rerank_model_name": "BAAI/bge-reranker-v2-m3"
}


In [29]:
# configure retriever
index = VectorStoreIndex.from_vector_store(vector_store, storage_context=storage_context)
vector_retriever = VectorIndexRetriever(
    index=index,
    vector_store_query_mode='mmr',
    similarity_top_k=cfg.retrieval_cfg.retrieval_dense_top_k,
    # sparse_top_k=cfg.retrieval_cfg.retrieval_sparse_top_k,
)

In [30]:
from llama_index.retrievers.bm25 import BM25Retriever
import Stemmer

bm25_retriever = BM25Retriever.from_defaults(
    nodes=nodes,
    similarity_top_k=cfg.retrieval_cfg.retrieval_sparse_top_k,
    stemmer=Stemmer.Stemmer("english"),
    language="english",
)

bm25_retrieval_results = bm25_retriever.retrieve("Nashville, grocery")
for result in bm25_retrieval_results:
    display_source_node(result, source_length=1000, show_source_metadata=True)

DEBUG [bm25s] Building index from IDs objects


**Node ID:** b0813f6c-6497-4218-9d80-5a2d6ce4b6ba<br>**Similarity:** 3.2125637531280518<br>**Text:** Convenient grocery store. A bit ghetto, and not because of it's location at all.<br>**Metadata:** {'review_id': '-kdFaer2K56NJCGNadTe5w', 'user_id': 'UYgAC9mvMYc12Ufc5KVUnA', 'business_id': '-4dYswJy7SPcbcERvitmIg', 'review_stars': 3, 'useful': 1, 'funny': 1, 'cool': 0, 'date': '2010-10-27T15:07:05', 'biz_name': 'Pathmark', 'biz_address': '3021 Grays Ferry Ave', 'biz_city': 'Philadelphia', 'biz_state': 'PA', 'biz_postal_code': '19146', 'biz_latitude': 39.9404026, 'biz_longitude': -75.1932966, 'biz_stars': 2.5, 'biz_review_count': 34, 'biz_is_open': 0, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': None, 'Ambience': None, 'BYOB': None, 'BestNights': None, 'BikeParking': 'True', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': None, 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': None, 'DriveThru': None, 'GoodForDancing': None, 'GoodForKids': None, 'GoodForMeal': None, 'HappyHour': None, 'HasTV': None, 'Music': None, 'NoiseLevel': None, 'OutdoorSeating': None, 'RestaurantsAttire': None, 'RestaurantsDelivery': None, 'RestaurantsGoodForGroups': None, 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': None, 'RestaurantsTableService': None, 'RestaurantsTakeOut': None, 'Smoking': None, 'WheelchairAccessible': None, 'WiFi': None}, 'biz_categories': ['Food', 'Grocery'], 'biz_hours': {'Friday': '6:0-1:0', 'Monday': '6:0-1:0', 'Saturday': '6:0-1:0', 'Sunday': '6:0-1:0', 'Thursday': '6:0-1:0', 'Tuesday': '6:0-1:0', 'Wednesday': '6:0-1:0'}}<br>

**Node ID:** 08d2c4b5-ef92-4f42-993b-3f8b16504530<br>**Similarity:** 3.1722183227539062<br>**Text:** A good grocery store, just requires a car to get to (or bus). This used to be somewhere they shuttled local university students from Penn to come get groceries; not sure if they still do that.

There's almost always a good deal to be had, fresh produce and meat, and feels like a truly suburban grocery store. Lines are long at times, but the only real problem is the distance from the city if you don't have a car.<br>**Metadata:** {'review_id': 'hKTTE9mOS1tXI6gDbDU_Mw', 'user_id': 'nnwBdqGHIAJQ5QX9lHOtrQ', 'business_id': '-4dYswJy7SPcbcERvitmIg', 'review_stars': 3, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2009-01-20T16:11:51', 'biz_name': 'Pathmark', 'biz_address': '3021 Grays Ferry Ave', 'biz_city': 'Philadelphia', 'biz_state': 'PA', 'biz_postal_code': '19146', 'biz_latitude': 39.9404026, 'biz_longitude': -75.1932966, 'biz_stars': 2.5, 'biz_review_count': 34, 'biz_is_open': 0, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': None, 'Ambience': None, 'BYOB': None, 'BestNights': None, 'BikeParking': 'True', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': None, 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': None, 'DriveThru': None, 'GoodForDancing': None, 'GoodForKids': None, 'GoodForMeal': None, 'HappyHour': None, 'HasTV': None, 'Music': None, 'NoiseLevel': None, 'OutdoorSeating': None, 'RestaurantsAttire': None, 'RestaurantsDelivery': None, 'RestaurantsGoodForGroups': None, 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': None, 'RestaurantsTableService': None, 'RestaurantsTakeOut': None, 'Smoking': None, 'WheelchairAccessible': None, 'WiFi': None}, 'biz_categories': ['Food', 'Grocery'], 'biz_hours': {'Friday': '6:0-1:0', 'Monday': '6:0-1:0', 'Saturday': '6:0-1:0', 'Sunday': '6:0-1:0', 'Thursday': '6:0-1:0', 'Tuesday': '6:0-1:0', 'Wednesday': '6:0-1:0'}}<br>

**Node ID:** ad6679fd-5acb-4cc2-a342-ea200ee88591<br>**Similarity:** 3.0762240886688232<br>**Text:** This grocery store SUCKS!  But it's the closest one to my house, so I go there for things that I need fast.  The produce is the worst possible.  I don't think the prices are very good, but then again it seems that all grocery stores in Philly are outrageously priced.  Employees are the slowest moving in the world!<br>**Metadata:** {'review_id': 'krw5FlAWXS16NkkqViK-wg', 'user_id': 'B-d6QdtkkoLHCyIyuT6y4A', 'business_id': '-4dYswJy7SPcbcERvitmIg', 'review_stars': 2, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2011-05-12T20:29:04', 'biz_name': 'Pathmark', 'biz_address': '3021 Grays Ferry Ave', 'biz_city': 'Philadelphia', 'biz_state': 'PA', 'biz_postal_code': '19146', 'biz_latitude': 39.9404026, 'biz_longitude': -75.1932966, 'biz_stars': 2.5, 'biz_review_count': 34, 'biz_is_open': 0, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': None, 'Ambience': None, 'BYOB': None, 'BestNights': None, 'BikeParking': 'True', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': None, 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': None, 'DriveThru': None, 'GoodForDancing': None, 'GoodForKids': None, 'GoodForMeal': None, 'HappyHour': None, 'HasTV': None, 'Music': None, 'NoiseLevel': None, 'OutdoorSeating': None, 'RestaurantsAttire': None, 'RestaurantsDelivery': None, 'RestaurantsGoodForGroups': None, 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': None, 'RestaurantsTableService': None, 'RestaurantsTakeOut': None, 'Smoking': None, 'WheelchairAccessible': None, 'WiFi': None}, 'biz_categories': ['Food', 'Grocery'], 'biz_hours': {'Friday': '6:0-1:0', 'Monday': '6:0-1:0', 'Saturday': '6:0-1:0', 'Sunday': '6:0-1:0', 'Thursday': '6:0-1:0', 'Tuesday': '6:0-1:0', 'Wednesday': '6:0-1:0'}}<br>

**Node ID:** 085de6f5-05b0-44b6-8720-6aa4e22e8d87<br>**Similarity:** 3.0317909717559814<br>**Text:** Free parking garage.  Eat in grocery store or deli type restaurant....everything is by the pound.<br>**Metadata:** {'review_id': 'vQxUaNanrYGZ_X3qCiIC3g', 'user_id': 'btAiOvMIfAbP5Sc5fAR8zA', 'business_id': 'lk9IwjZXqUMqqOhM774DtQ', 'review_stars': 4, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2017-07-08T18:07:49', 'biz_name': 'Caviar & Bananas', 'biz_address': '2031 Broadway', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37203', 'biz_latitude': 36.1483712, 'biz_longitude': -86.7988947, 'biz_stars': 3.5, 'biz_review_count': 159, 'biz_is_open': 0, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'beer_and_wine'", 'Ambience': "{'touristy': False, 'hipster': False, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': True, 'upscale': False, 'classy': True, 'casual': True}", 'BYOB': None, 'BestNights': "{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': False, 'thursday': False, 'sunday': False, 'saturday': False}", 'BikeParking': 'True', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': True, 'street': False, 'validated': False, 'lot': False, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': 'True', 'CoatCheck': 'False', 'Corkage': None, 'DogsAllowed': None, 'DriveThru': None, 'GoodForDancing': 'False', 'GoodForKids': 'True', 'GoodForMeal': "{'dessert': True, 'latenight': False, 'lunch': True, 'dinner': False, 'brunch': True, 'breakfast': True}", 'HappyHour': 'True', 'HasTV': 'False', 'Music': "{'dj': False, 'background_music': False, 'no_music': False, 'jukebox': False, 'live': False, 'video': False, 'karaoke': False}", 'NoiseLevel': "u'average'", 'OutdoorSeating': 'True', 'RestaurantsAttire': "'casual'", 'RestaurantsDelivery': 'True', 'RestaurantsGoodForGroups': 'True', 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': 'False', 'RestaurantsTableService': 'False', 'RestaurantsTakeOut': 'True', 'Smoking': "u'no'", 'WheelchairAccessible': 'True', 'WiFi': "'free'"}, 'biz_categories': ['Coffee & Tea', 'Restaurants', 'Wine Bars', 'Bars', 'Nightlife', 'American (Traditional)', 'Event Planning & Services', 'Food', 'Caterers', 'Breakfast & Brunch', 'Cafes', 'Diners'], 'biz_hours': {'Friday': '7:0-17:0', 'Monday': '7:0-17:0', 'Saturday': '7:0-17:0', 'Sunday': '7:0-17:0', 'Thursday': '7:0-17:0', 'Tuesday': '7:0-17:0', 'Wednesday': '7:0-17:0'}}<br>

...

**Node ID:** c741b868-3214-42f9-9146-a77a125f1625<br>**Similarity:** 2.1714556217193604<br>**Text:** Handcrafted ice cream right in the heart of Nashville. The place is always busy but worth the wait.<br>**Metadata:** {'review_id': '073JO4AkBwFz-6VoFSB--A', 'user_id': 'wa1PJGn1XxhIFbGWTeIbOQ', 'business_id': 'oaboaRBUgGjbo2kfUIKDLQ', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2019-07-07T22:57:17', 'biz_name': "Mike's Ice Cream", 'biz_address': '129 2nd Ave N', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37201', 'biz_latitude': 36.1626492, 'biz_longitude': -86.7759733, 'biz_stars': 4.5, 'biz_review_count': 593, 'biz_is_open': 1, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'none'", 'Ambience': "{'romantic': False, 'intimate': False, 'touristy': False, 'hipster': False, 'divey': False, 'classy': False, 'trendy': False, 'upscale': False, 'casual': True}", 'BYOB': None, 'BestNights': None, 'BikeParking': 'False', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}", 'ByAppointmentOnly': 'False', 'Caters': 'False', 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': 'False', 'DriveThru': 'False', 'GoodForDancing': None, 'GoodForKids': 'True', 'GoodForMeal': None, 'HappyHour': None, 'HasTV': 'False', 'Music': None, 'NoiseLevel': "u'average'", 'OutdoorSeating': 'None', 'RestaurantsAttire': "'casual'", 'RestaurantsDelivery': 'None', 'RestaurantsGoodForGroups': 'True', 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': 'False', 'RestaurantsTableService': None, 'RestaurantsTakeOut': 'True', 'Smoking': None, 'WheelchairAccessible': None, 'WiFi': "'free'"}, 'biz_categories': ['Ice Cream & Frozen Yogurt', 'Coffee & Tea', 'Restaurants', 'Sandwiches', 'Food'], 'biz_hours': {'Friday': '8:0-0:0', 'Monday': '8:0-23:0', 'Saturday': '8:0-0:30', 'Sunday': '8:0-23:0', 'Thursday': '8:0-23:0', 'Tuesday': '8:0-23:0', 'Wednesday': '8:0-23:0'}}<br>

**Node ID:** 2647fa86-79aa-42fb-a2e5-815fe782efb1<br>**Similarity:** 2.1714556217193604<br>**Text:** I had the Nashville Hot Chicken Salad and it was wonderful! Big enough to eat for lunch and dinner!<br>**Metadata:** {'review_id': 'R2AI3QGNuWNP2sHPd40ewg', 'user_id': '1xY7RkCcAzS-KBhZ0ZEE0w', 'business_id': 'lk9IwjZXqUMqqOhM774DtQ', 'review_stars': 4, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2019-10-23T18:24:14', 'biz_name': 'Caviar & Bananas', 'biz_address': '2031 Broadway', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37203', 'biz_latitude': 36.1483712, 'biz_longitude': -86.7988947, 'biz_stars': 3.5, 'biz_review_count': 159, 'biz_is_open': 0, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'beer_and_wine'", 'Ambience': "{'touristy': False, 'hipster': False, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': True, 'upscale': False, 'classy': True, 'casual': True}", 'BYOB': None, 'BestNights': "{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': False, 'thursday': False, 'sunday': False, 'saturday': False}", 'BikeParking': 'True', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': True, 'street': False, 'validated': False, 'lot': False, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': 'True', 'CoatCheck': 'False', 'Corkage': None, 'DogsAllowed': None, 'DriveThru': None, 'GoodForDancing': 'False', 'GoodForKids': 'True', 'GoodForMeal': "{'dessert': True, 'latenight': False, 'lunch': True, 'dinner': False, 'brunch': True, 'breakfast': True}", 'HappyHour': 'True', 'HasTV': 'False', 'Music': "{'dj': False, 'background_music': False, 'no_music': False, 'jukebox': False, 'live': False, 'video': False, 'karaoke': False}", 'NoiseLevel': "u'average'", 'OutdoorSeating': 'True', 'RestaurantsAttire': "'casual'", 'RestaurantsDelivery': 'True', 'RestaurantsGoodForGroups': 'True', 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': 'False', 'RestaurantsTableService': 'False', 'RestaurantsTakeOut': 'True', 'Smoking': "u'no'", 'WheelchairAccessible': 'True', 'WiFi': "'free'"}, 'biz_categories': ['Coffee & Tea', 'Restaurants', 'Wine Bars', 'Bars', 'Nightlife', 'American (Traditional)', 'Event Planning & Services', 'Food', 'Caterers', 'Breakfast & Brunch', 'Cafes', 'Diners'], 'biz_hours': {'Friday': '7:0-17:0', 'Monday': '7:0-17:0', 'Saturday': '7:0-17:0', 'Sunday': '7:0-17:0', 'Thursday': '7:0-17:0', 'Tuesday': '7:0-17:0', 'Wednesday': '7:0-17:0'}}<br>

**Node ID:** bf42fb80-364e-4d0e-bb6a-8c1697877300<br>**Similarity:** 2.162130832672119<br>**Text:** the cocktails were great & we ordered plenty of hot sake to enjoy our evening. the main reason why I am giving it 3 stars is because the menu is a little pricey (location is downtown nashville off 3rd) & nothing spectacular stood out for it being a new spot in nashville.<br>**Metadata:** {'review_id': 'OI6hq_7xWuK4JuFpkw_nqw', 'user_id': 'lU_H3xHRiWFkVStKX9sd9g', 'business_id': 'tMkwHmWFUEXrC9ZduonpTg', 'review_stars': 3, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2018-12-16T01:36:24', 'biz_name': 'The Green Pheasant', 'biz_address': '215 1st Ave S', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37201', 'biz_latitude': 36.1598858, 'biz_longitude': -86.7731974, 'biz_stars': 4.0, 'biz_review_count': 161, 'biz_is_open': 0, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'full_bar'", 'Ambience': "{'touristy': False, 'hipster': False, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': False, 'upscale': False, 'classy': True, 'casual': False}", 'BYOB': None, 'BestNights': None, 'BikeParking': 'False', 'BusinessAcceptsBitcoin': 'False', 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': True, 'street': False, 'validated': True, 'lot': False, 'valet': True}", 'ByAppointmentOnly': None, 'Caters': 'True', 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': 'False', 'DriveThru': None, 'GoodForDancing': None, 'GoodForKids': 'False', 'GoodForMeal': "{'dessert': None, 'latenight': None, 'lunch': None, 'dinner': True, 'brunch': None, 'breakfast': None}", 'HappyHour': 'True', 'HasTV': 'False', 'Music': None, 'NoiseLevel': "u'average'", 'OutdoorSeating': 'False', 'RestaurantsAttire': "u'casual'", 'RestaurantsDelivery': 'None', 'RestaurantsGoodForGroups': 'True', 'RestaurantsPriceRange2': None, 'RestaurantsReservations': 'True', 'RestaurantsTableService': 'True', 'RestaurantsTakeOut': 'None', 'Smoking': None, 'WheelchairAccessible': 'True', 'WiFi': "u'free'"}, 'biz_categories': ['Restaurants', 'Japanese', 'Seafood'], 'biz_hours': {'Friday': '16:0-23:0', 'Monday': None, 'Saturday': '16:0-23:0', 'Sunday': None, 'Thursday': '16:0-22:0', 'Tuesday': None, 'Wednesday': '16:0-22:0'}}<br>

**Node ID:** 990c9d76-4b85-406f-af78-894bdbb6b702<br>**Similarity:** 2.1461713314056396<br>**Text:** Stopped here on a whim for lunch on our way home to Orlando after a weekend at the beach. Delicious sandwiches, lasagna, ziti and salad. Bought some wonderful Italian groceries including fresh mozzarella  and fresh bread to take home. Very friendly place...family business.<br>**Metadata:** {'review_id': 'LC8mOm9RulAdyNlI9a2GNQ', 'user_id': 'UwVqHjSqTRz3FUgr2TPsKg', 'business_id': '0bPLkL0QhhPO5kt1_EXmNQ', 'review_stars': 5, 'useful': 1, 'funny': 0, 'cool': 0, 'date': '2015-12-30T20:36:26', 'biz_name': "Zio's Italian Market", 'biz_address': '2575 E Bay Dr', 'biz_city': 'Largo', 'biz_state': 'FL', 'biz_postal_code': '33771', 'biz_latitude': 27.9161159, 'biz_longitude': -82.7604608, 'biz_stars': 4.5, 'biz_review_count': 100, 'biz_is_open': 0, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'none'", 'Ambience': "{'romantic': False, 'intimate': False, 'touristy': False, 'hipster': False, 'divey': False, 'classy': False, 'trendy': False, 'upscale': False, 'casual': True}", 'BYOB': None, 'BestNights': None, 'BikeParking': 'True', 'BusinessAcceptsBitcoin': 'False', 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': 'True', 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': None, 'DriveThru': None, 'GoodForDancing': None, 'GoodForKids': 'True', 'GoodForMeal': "{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': False, 'brunch': False, 'breakfast': False}", 'HappyHour': None, 'HasTV': 'True', 'Music': None, 'NoiseLevel': "u'average'", 'OutdoorSeating': 'False', 'RestaurantsAttire': "u'casual'", 'RestaurantsDelivery': 'True', 'RestaurantsGoodForGroups': 'False', 'RestaurantsPriceRange2': '1', 'RestaurantsReservations': 'False', 'RestaurantsTableService': 'False', 'RestaurantsTakeOut': 'True', 'Smoking': None, 'WheelchairAccessible': 'True', 'WiFi': "u'no'"}, 'biz_categories': ['Food', 'Delis', 'Italian', 'Bakeries', 'Restaurants'], 'biz_hours': {'Friday': '10:0-20:0', 'Monday': '10:0-18:0', 'Saturday': '10:0-20:0', 'Sunday': None, 'Thursday': '10:0-20:0', 'Tuesday': '10:0-20:0', 'Wednesday': '10:0-20:0'}}<br>

**Node ID:** 4031e007-9327-4d5d-805a-ef67f8bb766f<br>**Similarity:** 2.142127275466919<br>**Text:** After visiting Mike's Ice Cream in Nashville, I am now saddened that my favorite spot is across the country. The home made Vanilla instantly reminded me of my youth when my dad used to make ice cream in the summer. If you are in Nashville, I highly recommend Mike's.<br>**Metadata:** {'review_id': 'rtUBWJgwUptv1fvulsGmnQ', 'user_id': 'E1Gfpm8X0JB-x0E7Gsa2qg', 'business_id': 'oaboaRBUgGjbo2kfUIKDLQ', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2012-06-01T21:50:16', 'biz_name': "Mike's Ice Cream", 'biz_address': '129 2nd Ave N', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37201', 'biz_latitude': 36.1626492, 'biz_longitude': -86.7759733, 'biz_stars': 4.5, 'biz_review_count': 593, 'biz_is_open': 1, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'none'", 'Ambience': "{'romantic': False, 'intimate': False, 'touristy': False, 'hipster': False, 'divey': False, 'classy': False, 'trendy': False, 'upscale': False, 'casual': True}", 'BYOB': None, 'BestNights': None, 'BikeParking': 'False', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}", 'ByAppointmentOnly': 'False', 'Caters': 'False', 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': 'False', 'DriveThru': 'False', 'GoodForDancing': None, 'GoodForKids': 'True', 'GoodForMeal': None, 'HappyHour': None, 'HasTV': 'False', 'Music': None, 'NoiseLevel': "u'average'", 'OutdoorSeating': 'None', 'RestaurantsAttire': "'casual'", 'RestaurantsDelivery': 'None', 'RestaurantsGoodForGroups': 'True', 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': 'False', 'RestaurantsTableService': None, 'RestaurantsTakeOut': 'True', 'Smoking': None, 'WheelchairAccessible': None, 'WiFi': "'free'"}, 'biz_categories': ['Ice Cream & Frozen Yogurt', 'Coffee & Tea', 'Restaurants', 'Sandwiches', 'Food'], 'biz_hours': {'Friday': '8:0-0:0', 'Monday': '8:0-23:0', 'Saturday': '8:0-0:30', 'Sunday': '8:0-23:0', 'Thursday': '8:0-23:0', 'Tuesday': '8:0-23:0', 'Wednesday': '8:0-23:0'}}<br>

In [31]:
# test_query = nodes[3].text
test_query = "Received confirmation of my order at 5 pm"
vector_results = vector_retriever.retrieve(test_query)
bm25_results = bm25_retriever.retrieve(test_query)

# TODO: Reconsider this test.
# Currently it aims to make sure the vector retriever, 
# when given a piece of text extracted from a node, 
# would return the original node.
# But what is the value of it?

# Check if the original node in top 10:
found = False
for vector_result in vector_results[:10]:
    target_id = bm25_results[0].id_
    if vector_result.id_ == target_id:
        found = True
        break
# assert found, "Can not find the original node in the top 10 vector results"
# assert bm25_results[0].id_ == vector_results[0].id_, "Different vector_retriever nodes vs bm25_retriever nodes"

In [32]:
from llama_index.core.retrievers import QueryFusionRetriever

# query_gen_prompt = """
# You are a helpful assistant that expands an input query into new strings that aim to increase the recall of an information retrieval system. The strings can be queries or paragraphs or sentences.
# You should apply different techniques to create new strings. Here are some example techniques:
# - Technique 1 - Optimize for full-text search: Extract the key concepts and entities as keywords. This technique can generate multiple strings instead of just one. Example input query: "What are some places to enjoy cold brew coffee in Hanoi?" -> Expected output:  string 1 = "cold brew coffee", string 2 = "hanoi". Do not modify the keyword, keep the keyword exactly in the input query. Return each output string on a new line.
# - Technique 2 - Optimize for similarity-based vector retrieval: Create a fake user review that should contain the answer for the question. Example input query: "What are some good Pho restaurants in Singapore?" -> Expected output query: "I found So Pho offerring a variety of choices to enjoy not Pho but some other Vietnamese dishes like bun cha. The price is reasonable."
# - Technique 3 - Increase recall: Paraphase the original question into a new question but use different synonyms or broader concepts. Example input query: "How to find some good local grocery stores near Changi Singapore?" -> Expected output query: "Where to find a good mart selling Singaporean stuffs to the east of Singapore?"

# Generate at least {num_queries} new strings by iterating over the technique in order. For example, your first generated string should always use technique 1, second technique 2, third technique 3. If run of of techniques then re-iterate from the start.

# Return one string on each line, related to the input query.

# Only return the strings. Never include the chosen technique.

# Input Query: {query}\n
# New strings:\n
# """

# query_gen_prompt = """
# You are a helpful assistant that expands an input query into new strings that aim to increase the recall of an information retrieval system. The strings can be queries or paragraphs or sentences.
# You should apply different techniques to create new strings. Here are some example techniques:
# - Technique 1 - Optimize for full-text search: Rephrase the input query to contain only important keywords. Remove all stopwords and low information words. Example input query: "What are some places to enjoy cold brew coffee in Hanoi?" -> Expected output:  "cold brew coffee hanoi"
# - Technique 2 - Optimize for similarity-based vector retrieval: Create a fake user review that should contain the answer for the question. Example input query: "What are some good Pho restaurants in Singapore?" -> Expected output query: "I found So Pho offerring a variety of choices to enjoy not Pho but some other Vietnamese dishes like bun cha. The price is reasonable."
# - Technique 3 - Increase recall: Paraphase the original question into a new question but use different synonyms or broader concepts. Example input query: "How to find some good local grocery stores near Changi Singapore?" -> Expected output query: "Where to find a good mart selling Singaporean stuffs to the east of Singapore?"

# Generate at least {num_queries} new strings by iterating over the technique in order. For example, your first generated string should always use technique 1, second technique 2, third technique 3. If run of of techniques then re-iterate from the start.

# Return one string on each line, related to the input query.

# Only return the strings. Never include the chosen technique.

# Input Query: {query}\n
# New strings:\n
# """

query_gen_prompt = """
You are a helpful assistant that expands an input query into new strings that aim to increase the recall of an information retrieval system. The strings can be queries or paragraphs or sentences.
You should apply different techniques to create new strings. Here are some example techniques:
- Technique 1 - Optimize for full-text search: Rephrase the input query to contain only important keywords. Remove all stopwords and low information words. Example input query: "What are some places to enjoy cold brew coffee in Hanoi?" -> Expected output:  "cold brew coffee hanoi"
- Technique 2 - Optimize for similarity-based vector retrieval: Create a fake user review that should contain the answer for the question. Example input query: "What are some good Pho restaurants in Singapore?" -> Expected output query: "I found So Pho offerring a variety of choices to enjoy not Pho but some other Vietnamese dishes like bun cha. The price is reasonable."

Generate at least {num_queries} new strings by iterating over the technique in order. For example, your first generated string should always use technique 1, second technique 2. If run of of techniques then re-iterate from the start.

Return one string on each line, related to the input query.

Only return the strings. Never include the chosen technique.

Input Query: {query}\n
New strings:\n
"""

from llama_index.llms.openai import OpenAI
llm = OpenAI(model=cfg.eval_cfg.response_eval_llm_model, **cfg.eval_cfg.response_eval_llm_model_config)

retriever = QueryFusionRetriever(
    [vector_retriever, bm25_retriever],
    llm=llm,
    similarity_top_k=cfg.retrieval_cfg.retrieval_top_k,
    num_queries=2,  # set this to 1 to disable query generation
    mode="reciprocal_rerank",
    use_async=True,
    verbose=True,
    query_gen_prompt=query_gen_prompt
)

## Test retrieval

In [33]:
query = "What are some places to enjoy cold brew coffee"
retrieval_results = await retriever.aretrieve(query)
for node in retrieval_results:
    display_source_node(node, source_length=1000)
    print(node.get_content(metadata_mode=MetadataMode.LLM))
    print("\n")

Generated queries:
cold brew coffee places
"I recently visited a cozy café that specializes in cold brew coffee, and I was blown away by their unique flavors and smooth texture. It's the perfect spot to relax and enjoy a refreshing drink!"


**Node ID:** 30bd460d-02f9-4439-bcc7-c0739243a2c0<br>**Similarity:** 0.06293747829742127<br>**Text:** I'm a fan of dark roast but it's usually not available. My advice brew different flavors of coffee and keep it hot.<br>

business_id: ppFCk9aQkM338Rgwpl2F5A
review_stars: 4
biz_name: Wawa
biz_address: 3604 Chestnut St
biz_city: Philadelphia
biz_state: PA
biz_categories: ['Restaurants', 'Automotive', 'Delis', 'Gas Stations', 'Food', 'Coffee & Tea', 'Sandwiches', 'Convenience Stores']

I'm a fan of dark roast but it's usually not available. My advice brew different flavors of coffee and keep it hot.




**Node ID:** b0d9bac3-56a7-4b66-9654-130a8e680efd<br>**Similarity:** 0.06164584421323263<br>**Text:** If it's cold, come grab a coffee instead. They offer all the great styles and flavors of coffee as any other coffee house you'll find.<br>

business_id: oaboaRBUgGjbo2kfUIKDLQ
review_stars: 5
biz_name: Mike's Ice Cream
biz_address: 129 2nd Ave N
biz_city: Nashville
biz_state: TN
biz_categories: ['Ice Cream & Frozen Yogurt', 'Coffee & Tea', 'Restaurants', 'Sandwiches', 'Food']

If it's cold, come grab a coffee instead. They offer all the great styles and flavors of coffee as any other coffee house you'll find.




...

business_id: IDtLPgUrqorrpqSLdfMhZQ
review_stars: 5
biz_name: Helena Avenue Bakery
biz_address: 131 Anacapa St, Ste C
biz_city: Santa Barbara
biz_state: CA
biz_categories: ['Food', 'Restaurants', 'Salad', 'Coffee & Tea', 'Breakfast & Brunch', 'Sandwiches', 'Bakeries']

Got a breakfast sandwich. Biscuit with egg, bacon and cheese. It comes with greens and some tomato spread, i didn't want that. It was delicious. Cold brew decent also. Cashiers were sweet and accommodating. The girl serving the food had a flat affect. Ambiance is a chill, open, industrial vibe. I wish the outdoor space was larger. There is a wine tasting area in the same spot. Would come here again just for that biscuit!




**Node ID:** cfb9f5e5-4c7d-44e9-b245-585ce764254e<br>**Similarity:** 0.0220205686630369<br>**Text:** After enjoying their current lineup, I walked around the relaxed tap room, decked out with couches and lounging chairs. Oh, and if you want further proof that Bier is about quality in addition to quantity, make sure you check out their wall of fame, covered in numerous awards and ribbons from various brewing competitions, including the Great American Beer Festival. 

Just like one of their self-proclaimed/named brews, PDG, this place is pretty damn good. 'Nuff said. 4.5 Stars.<br>

business_id: 7clCBgNbd-x2Wj96lZ6Mjw
review_stars: 4
biz_name: Bier Brewery and Tap Room
biz_address: 5133 E 65th St
biz_city: Indianapolis
biz_state: IN
biz_categories: ['Food', 'Beer', 'Wine & Spirits', 'Breweries']

After enjoying their current lineup, I walked around the relaxed tap room, decked out with couches and lounging chairs. Oh, and if you want further proof that Bier is about quality in addition to quantity, make sure you check out their wall of fame, covered in numerous awards and ribbons from various brewing competitions, including the Great American Beer Festival. 

Just like one of their self-proclaimed/named brews, PDG, this place is pretty damn good. 'Nuff said. 4.5 Stars.




**Node ID:** 827c5590-a280-48f4-8c5f-f20b79406292<br>**Similarity:** 0.021968787515006<br>**Text:** The boyfriend and I adore coffee in a can't-function-without-it-in-the-morning kind of a way. We are always on the lookout for great coffee places and Roast definitely falls into that category! 

We were both pleasantly surprised to find a maple bacon latte on the menu and of course the boyfriend had to try it. I was also happy to see wine (and Canadian wine at that) on their menu. Canadian wines are typically some of the best wines around! 

Roast also introduced me to a great new rewards program, Perka! I was happy to learn that after 6 coffees you receive a free drink! This rewards program trumps any Starbucks gold star reward program. 

I will definitely be back to this great little coffee house!<br>

business_id: WKMJwqnfZKsAae75RMP6jA
review_stars: 5
biz_name: Roast Coffeehouse and Wine Bar
biz_address: 10359 104 Street NW
biz_city: Edmonton
biz_state: AB
biz_categories: ['Coffee & Tea', 'Food', 'Cafes', 'Bars', 'Wine Bars', 'Restaurants', 'Nightlife']

The boyfriend and I adore coffee in a can't-function-without-it-in-the-morning kind of a way. We are always on the lookout for great coffee places and Roast definitely falls into that category! 

We were both pleasantly surprised to find a maple bacon latte on the menu and of course the boyfriend had to try it. I was also happy to see wine (and Canadian wine at that) on their menu. Canadian wines are typically some of the best wines around! 

Roast also introduced me to a great new rewards program, Perka! I was happy to learn that after 6 coffees you receive a free drink! This rewards program trumps any Starbucks gold star reward program. 

I will definitely be back to this great little coffee house!




## Retrieval Evaluation

### Synthetic

In [34]:
from src.run.eval import RetrievalEvaluator

In [35]:
retrieval_evaluator = RetrievalEvaluator()
retrieval_evaluator.generate_synthetic_dataset(cfg, nodes)

2024-08-13 21:23:56.111 | INFO     | src.run.eval.retrieval:generate_synthetic_dataset:36 - Creating new retrieval eval dataset at data/026_rez_tool/retrieval_synthetic_eval_dataset.json...
2024-08-13 21:23:56.112 | INFO     | src.run.eval.retrieval:generate_synthetic_dataset:40 - Sampling 30 nodes for retrieval evaluation...
2024-08-13 21:23:56.124 | INFO     | src.run.eval.retrieval:generate_synthetic_dataset:67 - Creating new synthetic retrieval eval dataset...
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:23<00:00,  1.30it/s]
2024-08-13 21:24:19.186 | INFO     | src.run.eval.retrieval:generate_synthetic_dataset:74 - Persisting synthetic retrieval eval dataset to data/026_rez_tool/retrieval_synthetic_eval_dataset.json...


In [36]:
retrieval_evaluator.retrieval_eval_dataset.queries

{'e81ecf1f-4f20-49e9-bb9b-a966c9ff3a3c': 'What are some restaurants with a lively atmosphere and diverse menu options?',
 'ede13d12-b774-4b81-b3ed-a23c5a20df1c': 'What are some reliable shipping or delivery services that ensure items are received without damage?',
 '8a04f5c2-bb62-46c2-8f6d-e28316ed380a': 'What are some affordable parking options in Midtown Nashville?',
 '9b730abd-968b-4d4a-bbe3-8484023e84ba': 'What are some businesses or services that have received negative reviews recently?',
 '1950cfa7-6e9e-4fc1-98a3-5286c77dd82a': 'What are some ice cream shops that offer a cozy atmosphere during winter?',
 '4ed6a424-5fd1-4af1-90f0-d89bf1afba59': 'What are some popular ice cream shops in Nashville that offer waffle cones?',
 '3c5a57a5-a058-4be2-8950-f8efcbcd4e9d': 'What are some sushi restaurants that offer a la carte options instead of tasting menus?',
 '7d40337f-5ebc-4bb4-a857-cdbadf593cdf': 'What are some cafes known for their excellent breakfast sandwiches and coffee?',
 '3b96c3

In [37]:
retrieval_eval_results_df, retrieval_eval_results_full_df = await retrieval_evaluator.aevaluate(cfg, retriever)

Generated queries:
lively atmosphere diverse menu restaurants
"I recently dined at a restaurant that had an incredible vibe, perfect for a night out. The menu was diverse, offering everything from international cuisine to local favorites, making it a great choice for groups with different tastes."
Generated queries:
reliable shipping delivery services items received undamaged
"I recently used FastShip for my deliveries, and I was impressed by their careful handling of packages. Everything arrived in perfect condition, and their customer service was very responsive."
Generated queries:
affordable parking options midtown nashville
"I recently visited Midtown Nashville and found several affordable parking spots. The parking garage on 21st Avenue was a great deal, and I also discovered street parking near the restaurants that didn't cost much."
Generated queries:
negative reviews businesses services recent
Generated queries:
ice cream shops cozy atmosphere winter
"I visited Creamy Delights

In [38]:
retrieval_eval_results_df

,retrievers,hit_rate,mrr,precision,recall,ap,ndcg
0,top_50_retrieval_eval,0.7,0.258254,0.014,0.7,0.258254,0.02776


In [39]:
retrieval_eval_results_full_df

,query,expected_ids,retrieved_ids,hit_rate,mrr,precision,recall,ap,ndcg
0,What are some restaurants with a lively atmosp...,[1c5997af-1609-4da3-a7b8-41314274b8c0],"[2dda015a-5910-4855-9ac6-3a90d3752b00, 8c3f397...",1.0,0.083333,0.02,1.0,0.083333,0.020952
1,What are some reliable shipping or delivery se...,[688ebed3-03e8-49ac-816d-c9e67ab9dc55],"[811021c9-b981-49f8-9675-4ba2426eb9aa, b88c7f8...",1.0,0.125000,0.02,1.0,0.125000,0.024459
2,What are some affordable parking options in Mi...,[11199ffc-14e7-4ba5-bb01-ee241d7ebc79],"[41c651f3-c362-45e2-a23a-45bc37b8423e, 11199ff...",1.0,0.500000,0.02,1.0,0.500000,0.048918
3,What are some businesses or services that have...,[0be365a1-9a4c-4eb0-8b53-6c0b609d272d],"[9a73f405-1c6b-4eda-b3d0-3934a4752cdf, 7057d3b...",0.0,0.000000,0.00,0.0,0.000000,0.000000
4,What are some ice cream shops that offer a coz...,[30100df1-b6f4-44cd-b8b5-06753f52f013],"[e595dcff-ae9e-4d5e-9563-f7d05770efa1, a46dcd3...",1.0,0.047619,0.02,1.0,0.047619,0.017386
5,What are some popular ice cream shops in Nashv...,[799ee664-933b-4d76-bf91-2c76830d343c],"[060d1490-f877-4a10-852a-abc664a36049, 799ee66...",1.0,0.500000,0.02,1.0,0.500000,0.048918
6,What are some sushi restaurants that offer a l...,[1a9a1c9b-dc13-4d70-9842-41c71bb6288b],"[f3bb4984-8437-4591-8b2f-dd5d4d27f7cd, 1a9a1c9...",1.0,0.500000,0.02,1.0,0.500000,0.048918
7,What are some cafes known for their excellent ...,[4313ee69-19af-40c4-839c-4fee6168f27d],"[a892cc58-7344-4b94-b70f-d74160025232, be4a661...",1.0,0.200000,0.02,1.0,0.200000,0.029994
8,What are some financial institutions known for...,[a2dcbe33-6b09-42c0-9800-d600b473df86],"[bce76706-6fad-4e00-a97a-2ea9e749cc30, 87afa19...",0.0,0.000000,0.00,0.0,0.000000,0.000000
9,What are some restaurants that are known for t...,[27bb740f-b16d-4887-83da-e296f2c20871],"[fb377197-b7c3-4909-a95d-32a6a927be76, e8f7673...",1.0,0.333333,0.02,1.0,0.333333,0.038767


#### Error Analysis

In [40]:
retrieval_eval_irrelevance_df = (
    retrieval_eval_results_full_df
    .loc[lambda df: df['hit_rate'].lt(1)]
    .sort_values(['hit_rate', 'mrr', 'precision', 'recall', 'ap', 'ndcg'])
)
with pd.option_context('display.max_colwidth', 400):
    display(retrieval_eval_irrelevance_df)

,query,expected_ids,retrieved_ids,hit_rate,mrr,precision,recall,ap,ndcg
3,What are some businesses or services that have received negative reviews recently?,[0be365a1-9a4c-4eb0-8b53-6c0b609d272d],"[9a73f405-1c6b-4eda-b3d0-3934a4752cdf, 7057d3bf-4dfd-4831-97d6-0bd3718b6607, b4d58382-b108-4896-a1bc-9d746615d5fc, 8a946ceb-bc01-4761-87da-7c19166c15b2, 317b9bdb-0545-41e1-98ae-0891eec13993, d7464c44-20c2-492e-b4f7-d3b70455d5ea, 016aaa6c-92a2-4063-9df7-cc7d80c15600, f5ff1d51-50ac-4f2b-89a3-18993e140ab2, 3597fa8e-cb8a-4099-9e85-5126f303a1ae, eec28b12-7506-4af7-84d7-a0c493f4cfb2, d8ab56d8-20d2-4...",0.0,0.0,0.0,0.0,0.0,0.0
8,What are some financial institutions known for their customer service and reliability?,[a2dcbe33-6b09-42c0-9800-d600b473df86],"[bce76706-6fad-4e00-a97a-2ea9e749cc30, 87afa191-708a-4948-a2d3-991ff5ea052f, 83bf3b8d-8985-4c67-8424-9eaa0e26c53d, 39b1cb8b-357a-41ff-b93d-9ed3ad6c0f81, b6e89800-fc67-4e45-b7f0-7dd37aea3d67, 99c41fee-dc0c-4848-b5d2-d3056e2c326c, 5e57ac84-3a51-43d4-af3a-4bb5dac18fa6, 66c01de4-b58c-4279-b22e-4c39e18eba11, 80ac3eea-94c1-4284-987b-9e0d2c610cc0, 7bcaf6cd-4fa8-4f1e-9676-9b37feee42d0, 0087308e-2b1b-4...",0.0,0.0,0.0,0.0,0.0,0.0
11,What are some ice cream shops that offer a variety of flavors and make their own waffle cones?,[72712c9f-5757-46ac-bc9e-b3c20e61c538],"[30e0b393-c018-4fe4-9d1c-2a9f6304462a, 802772ab-8608-4308-982e-3867f7116e0e, ba8638ce-29f2-4a49-bcd0-498f8062a08c, f1b5244d-ee90-42aa-b52e-e6c72e37734e, ce585ed1-16fd-4e1a-a891-3d98542ce846, 891fcb7f-3e67-4d99-a008-0547a1feab1f, 2c06183f-d077-40fd-ab29-86b98709a998, 877935ef-7f49-4aa0-903b-61e62ecda26c, 1b7ceb97-f3af-4053-b680-6de28244b0fa, df260e7b-c7a8-4a1e-a0da-fd1dd917ed4e, 060d1490-f877-4...",0.0,0.0,0.0,0.0,0.0,0.0
12,What are some popular shopping destinations for clothing and accessories?,[affc6819-a59e-46b2-a292-97aded224c43],"[a6cda44f-b899-4a17-b457-f78e318b18be, a191d388-2d85-4b99-886d-9c80b7cd20fd, 1f91f5a1-e97b-4d99-bc02-9f5fa3d724b3, 26159980-b130-4a6f-b51b-5cf1935936e8, 77b408e6-39f0-4e55-9ecd-88fd19ad1c37, fd6eb5e3-94b0-4ab5-982c-ad8e657ccde2, aec6caf2-bb6e-4e4d-bcde-4e4c82383621, 2ebed090-ac19-44de-88f4-c72f7f3f6d63, 489e5c14-1517-478d-ba93-350dd415932a, ac5646b7-9c01-4d04-9eb1-9cdeb545b5c3, 8856f366-7799-4...",0.0,0.0,0.0,0.0,0.0,0.0
16,What are some popular attractions or places to visit in the area?,[c661a2d4-483b-4f4e-98b2-5ea092d5d007],"[b3b5cd6f-3d60-480b-b2ba-35fc769836a6, e5666eb7-e74f-44e8-b872-26ac55b596f4, b613e3bf-80e5-4157-bb5f-6ec1fe27157e, 354628cf-ddbf-46d8-9425-c334556ce1f9, c2656a03-6565-4a8a-9977-fc81891b0e7e, 0ef88895-cd29-45b1-9dfe-6bc837dc95e3, 9da32269-619b-4a76-92eb-cb17a6a96dd0, c2345bab-253b-4013-a403-c2aa586cb901, 10c5f408-ddd5-4b41-89bd-419ce67267e9, 8724034e-32a5-4bf7-84c6-1eac3ebf62fc, c72b9f83-4f58-4...",0.0,0.0,0.0,0.0,0.0,0.0
17,What are some engaging activities or attractions for children that would leave a lasting impression?,[ec48e41f-2922-40ca-818e-bbcc9884e4d9],"[8a680b53-0cbe-4441-bd48-daafcde4bacc, ed529a7a-a8b7-42de-842b-9aeeff32d0b1, 29dcdd0c-7c34-4cfc-a70e-c108a5958c3c, 47b2cdb2-168e-4442-944e-709724aa2259, 3594d0f9-8b8a-48d2-a6ab-fcfd88480efa, 7935b118-016f-4d8c-b761-ee76513296a8, 7a7031b0-ac5f-4500-a03c-8588a5b6656f, 7b307531-b19c-4f83-bf3f-f8f11f86286b, 73493d80-b4da-4748-89a8-bdb9ae41b755, b60fda71-232f-425d-9695-741aafc1335b, e9b67928-f455-4...",0.0,0.0,0.0,0.0,0.0,0.0
21,What are some restaurants known for their exceptional customer service and knowledgeable staff?,[8be2e301-e805-4d23-a9d4-e8fc4c5a8abd],"[6fcb2ddb-bb7f-4cb9-827e-281024163863, e8f7673e-3b4b-4bc3-a63c-f55af5ab29f0, c9f022de-5cb8-4407-943a-933e28678ee4, 0b2ca853-5e04-4ea8-9336-0088c4b6ea52, 23fc9df8-7910-496f-bf00-371ca3d25367, dfa73ec3-4dd7-4c00-bd78-7cd8174e3a52, 59b43095-15d0-4152-811a-af8c12436579, 897efb82-ae2c-41ec-80dd-294339295261, 386c8a8c-4d8d-425b-855c-2957397bfebe, c2dbc778-88fd-40e8-a775-a310d53213b6, 39d5a81b-943e-4...",0.0,0.0,0.0,0.0,0.0,0.0
22,What are some popular tasting rooms or wine bars 

In [41]:
for i, row in retrieval_eval_irrelevance_df.reset_index(drop=True).iterrows():
    print(f"\n\n============Error #{i+1}=============\n\n")
    print(f"Query:\n{row.query}\n")
    print(f"- Expected contexts:\n")
    expected_nodes = docstore.get_nodes(row.expected_ids)
    for node in expected_nodes:
        print(node.get_content(metadata_mode=MetadataMode.LLM))

    print(f"\n- Retrieved contexts:\n")
    retrieved_nodes = docstore.get_nodes(row.retrieved_ids)
    for i, node in enumerate(retrieved_nodes[:10]):
        print(f"+ Context #{i+1}:\n\n{node.get_content(metadata_mode=MetadataMode.LLM)}\n")



============Error #1=============


Query:
What are some businesses or services that have received negative reviews recently?

- Expected contexts:

business_id: noByYNtDLQAra9ccqxdfDw
review_stars: 1
biz_name: H&M
biz_address: 827-833 State St
biz_city: Santa Barbara
biz_state: CA
biz_categories: ["Women's Clothing", 'Accessories', "Children's Clothing", "Men's Clothing", 'Adult', 'Shopping', 'Fashion']

Awful.

- Retrieved contexts:

+ Context #1:

business_id: 7PDi_iyik3jraDAzWwwR4Q
review_stars: 1
biz_name: Chase JP Morgan Bank Credit Card Services
biz_address: 
biz_city: Wilmington
biz_state: DE
biz_categories: ['Banks & Credit Unions', 'Financial Services']

That is NOT how you keep clients happy! But judging by all of negative reviews here, I doubt someone would read this or care enough about the reviews to do anything about it. 
You have been warned! Stay away from Chase!

+ Context #2:

business_id: 7PDi_iyik3jraDAzWwwR4Q
review_stars: 1
biz_name: Chase JP Morgan Bank Credit

In [42]:
def check_retrieval_results(query):
    retrieval_results = retriever.retrieve(query)
    for node in retrieval_results:
        display_source_node(node, source_length=1000)
        print(node.get_content(metadata_mode=MetadataMode.EMBED))
        print("\n")
    return retrieval_results

query = "What are some good grocery stores in Nashville that offer curated items and unique local products?"
check_retrieval_results(query)

Generated queries:
grocery stores nashville curated items local products
"I recently visited a grocery store in Nashville that had an amazing selection of curated items and unique local products. They really focus on supporting local farmers and artisans, making it a great place to find something special."


**Node ID:** 11199ffc-14e7-4ba5-bb01-ee241d7ebc79<br>**Similarity:** 0.06124571945467468<br>**Text:** Interior is gorgeous. Very clean and modern. It'll be great for a lot of Nashville and tourist Instagrammers to get their pose on. 

As far as groceries, lots of curated items from both Nashville and beyond. They have McClures pickles... being from Michigan I got super pumped about those. They also carry Jenis Ice Cream pints which are down right dangerous. Great selection of coffee, sweets, cheeses, charcuterie and wine.

The only reason it's not a full 5 is the parking situation. All the lots in midtown are super expensive or you need change for meters (it's 2017 I don't carry anything that's monetary unless it's a card). If you do meters, it's $1.50 per hour in that area.  Paid lots are about $8 an hour.<br>

business_id: lk9IwjZXqUMqqOhM774DtQ
review_stars: 4
biz_name: Caviar & Bananas
biz_address: 2031 Broadway
biz_city: Nashville
biz_state: TN
biz_categories: ['Coffee & Tea', 'Restaurants', 'Wine Bars', 'Bars', 'Nightlife', 'American (Traditional)', 'Event Planning & Services', 'Food', 'Caterers', 'Breakfast & Brunch', 'Cafes', 'Diners']

Interior is gorgeous. Very clean and modern. It'll be great for a lot of Nashville and tourist Instagrammers to get their pose on. 

As far as groceries, lots of curated items from both Nashville and beyond. They have McClures pickles... being from Michigan I got super pumped about those. They also carry Jenis Ice Cream pints which are down right dangerous. Great selection of coffee, sweets, cheeses, charcuterie and wine.

The only reason it's not a full 5 is the parking situation. All the lots in midtown are super expensive or you need change for meters (it's 2017 I don't carry anything that's monetary unless it's a card). If you do meters, it's $1.50 

**Node ID:** ef4f9c18-752a-4eae-8367-9b506155d237<br>**Similarity:** 0.06069231211546202<br>**Text:** This is a very interesting spot for specialty items and local items like that delicious Nashville fountain of juice. This is a great place to grab coffee, dessert or a salad / sandwich for lunch.<br>

business_id: lk9IwjZXqUMqqOhM774DtQ
review_stars: 4
biz_name: Caviar & Bananas
biz_address: 2031 Broadway
biz_city: Nashville
biz_state: TN
biz_categories: ['Coffee & Tea', 'Restaurants', 'Wine Bars', 'Bars', 'Nightlife', 'American (Traditional)', 'Event Planning & Services', 'Food', 'Caterers', 'Breakfast & Brunch', 'Cafes', 'Diners']

This is a very interesting spot for specialty items and local items like that delicious Nashville fountain of juice. This is a great place to grab coffee, dessert or a salad / sandwich for lunch.




...

**Node ID:** 1b4fe863-f992-4e90-86bf-7da0292a0674<br>**Similarity:** 0.022592592592592595<br>**Text:** Consider yourself warned: The variety of ice cream at this place may cause your head to implode with indecision.

During CMA Fest when we went, this place was lined up out the door almost constantly. I'm sure it's not like that all the time, but even if it is, it's worth the wait.

The ice cream tastes super fresh and there are some really unique flavors to choose from. I got a double scoop - Tennessee fudge and Take That! Both were delicious; TT had what I identified as Whoppers in it, while Tennessee fudge is pretty self-explanatory. I preferred the Tennessee fudge, however.<br>

business_id: oaboaRBUgGjbo2kfUIKDLQ
review_stars: 4
biz_name: Mike's Ice Cream
biz_address: 129 2nd Ave N
biz_city: Nashville
biz_state: TN
biz_categories: ['Ice Cream & Frozen Yogurt', 'Coffee & Tea', 'Restaurants', 'Sandwiches', 'Food']

Consider yourself warned: The variety of ice cream at this place may cause your head to implode with indecision.

During CMA Fest when we went, this place was lined up out the door almost constantly. I'm sure it's not like that all the time, but even if it is, it's worth the wait.

The ice cream tastes super fresh and there are some really unique flavors to choose from. I got a double scoop - Tennessee fudge and Take That! Both were delicious; TT had what I identified as Whoppers in it, while Tennessee fudge is pretty self-explanatory. I preferred the Tennessee fudge, however.




**Node ID:** ee507900-2e0f-4cee-b794-27c68f457a93<br>**Similarity:** 0.02250368912936547<br>**Text:** This was my first trip back and they did not disappoint.  Their asian tofu....delicious.  And when you add to to one of their incredibly fresh salads...one of my favorites anywhere.  My friend tried the sandwich of the day, smoked salmon, and loved it.

Please welcome them back with open arms...make multiple trips.  Nashville needs a spot like this and I'm so grateful they gave us another chance, let's not let them down!<br>

business_id: lk9IwjZXqUMqqOhM774DtQ
review_stars: 5
biz_name: Caviar & Bananas
biz_address: 2031 Broadway
biz_city: Nashville
biz_state: TN
biz_categories: ['Coffee & Tea', 'Restaurants', 'Wine Bars', 'Bars', 'Nightlife', 'American (Traditional)', 'Event Planning & Services', 'Food', 'Caterers', 'Breakfast & Brunch', 'Cafes', 'Diners']

This was my first trip back and they did not disappoint.  Their asian tofu....delicious.  And when you add to to one of their incredibly fresh salads...one of my favorites anywhere.  My friend tried the sandwich of the day, smoked salmon, and loved it.

Please welcome them back with open arms...make multiple trips.  Nashville needs a spot like this and I'm so grateful they gave us another chance, let's not let them down!




[NodeWithScore(node=TextNode(id_='11199ffc-14e7-4ba5-bb01-ee241d7ebc79', embedding=None, metadata={'review_id': 'VZMjK-Moy-fN08mR2m9JUg', 'user_id': 'U5hfpmnqW65_u10C1i06Dw', 'business_id': 'lk9IwjZXqUMqqOhM774DtQ', 'review_stars': 4, 'useful': 4, 'funny': 1, 'cool': 0, 'date': '2017-06-23T12:16:51', 'biz_name': 'Caviar & Bananas', 'biz_address': '2031 Broadway', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37203', 'biz_latitude': 36.1483712, 'biz_longitude': -86.7988947, 'biz_stars': 3.5, 'biz_review_count': 159, 'biz_is_open': 0, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'beer_and_wine'", 'Ambience': "{'touristy': False, 'hipster': False, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': True, 'upscale': False, 'classy': True, 'casual': True}", 'BYOB': None, 'BestNights': "{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': False, 'thursday': False, 'sunday': False, 'saturday': False}", 'BikeParking': 'True', 'BusinessAcc

# Response

In [43]:
from llama_index.core import get_response_synthesizer
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.postprocessor.flag_embedding_reranker import FlagEmbeddingReranker
from llama_index.core.response_synthesizers.type import ResponseMode

from src.features.append_reference.custom_query_engine import ManualAppendReferenceQueryEngine
from src.features.synthesize_recommendation.custom_tree_summarize import CUSTOM_TREE_SUMMARIZE_PROMPT_SEL

In [44]:
node_postprocessors = []

if cfg.retrieval_cfg.retrieval_similarity_cutoff is not None:
    node_postprocessors.append(SimilarityPostprocessor(similarity_cutoff=cfg.retrieval_cfg.retrieval_similarity_cutoff))

reranker = FlagEmbeddingReranker(model=cfg.retrieval_cfg.rerank_model_name, top_n=cfg.retrieval_cfg.rerank_top_k, use_fp16=True)
node_postprocessors.append(reranker)

response_synthesizer = get_response_synthesizer(
    response_mode=ResponseMode.TREE_SUMMARIZE,
    summary_template=CUSTOM_TREE_SUMMARIZE_PROMPT_SEL
)
query_engine = ManualAppendReferenceQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=node_postprocessors,
)

## Test Query Engine

In [45]:
from llama_index.core.response.notebook_utils import (
    display_source_node,
    display_response,
)

In [46]:
# query = "What are some gastropubs in Indianapolis that are known for their cocktails?"
query = "What are some places to enjoy cold brew coffee"
response = query_engine.query(query)
display_response(response, source_length=1000, show_source=True, show_metadata=True, show_source_metadata=True)

Generated queries:
cold brew coffee places
"I recently visited a cozy café that specializes in cold brew coffee. The atmosphere was perfect for relaxing, and they had a fantastic selection of cold brew options that really hit the spot!"


**`Final Response:`** Based on your interest in enjoying cold brew coffee, here are several recommendations ranked by relevance:

### 1. **Helena Avenue Bakery**
   - **Location:** 131 Anacapa St, Ste C, Santa Barbara, CA
   - **Review Stars:** 4-5
   - **Strengths:**
     - Multiple reviews highlight the quality of their cold brew coffee, describing it as "so good" and "decent."
     - The bakery offers a variety of delicious breakfast options, including sandwiches and pastries, making it a great spot for a meal alongside your coffee.
     - The ambiance is described as chill and open, with a pleasant outdoor patio for enjoying your drink.
   - **Comparison:** This is the top recommendation due to the consistent praise for both the cold brew and the overall food experience, making it ideal for a leisurely breakfast or brunch.

### 2. **Mike's Ice Cream**
   - **Location:** 129 2nd Ave N, Nashville, TN
   - **Review Stars:** 4-5
   - **Strengths:**
     - Offers a variety of coffee options, including cold brew, alongside their diverse ice cream flavors.
     - The atmosphere is described as a great place to chill, making it suitable for a casual coffee outing.
     - Friendly staff and quick service enhance the experience.
   - **Comparison:** While Mike's Ice Cream is a solid choice for cold brew, it is slightly less focused on coffee than Helena Avenue Bakery, which specializes in breakfast and baked goods. However, if you're in the mood for a sweet treat with your coffee, this could be a fun option.

### 3. **Caviar & Bananas**
   - **Location:** 2031 Broadway, Nashville, TN
   - **Review Stars:** 4
   - **Strengths:**
     - Offers a variety of beverages, including cold brew coffee, and is a versatile spot for food, wine, and gifts.
     - The place is suitable for informal meet-ups, making it a good option if you're looking to socialize while enjoying your coffee.
   - **Comparison:** Caviar & Bananas is a more diverse venue compared to the others, but it may not provide the same focused coffee experience as Helena Avenue Bakery or Mike's Ice Cream. It’s ideal if you want a broader selection of food and drink options.

### Summary
- **Helena Avenue Bakery** is the best choice for a dedicated coffee and breakfast experience, especially for cold brew lovers.
- **Mike's Ice Cream** is great if you want a casual atmosphere with the option of ice cream, making it a fun spot for a treat.
- **Caviar & Bananas** offers a wider range of options but may not be as coffee-centric as the other two. 

Choose based on whether you prefer a focused coffee experience, a casual treat, or a versatile venue for food and drinks!


#### Referenced Paragraphs
**Helena Avenue Bakery**

> Pricey ( a ham and cheese croissant was $5.50)  as with anywhere in Santa Barbra but very delicious. Cute space ( a bit hard to find ). The green eggs and ham  breakfast biscuit is to die for.  ($10 I think) The Santa Barbra tri tip sandwich was excellent! Cold brew coffee was good. Lots of good choices in the grab and go section, and our salami and pesto sammy was also great.  (Review ID: <REVIEW_ID>Q3fPo_x6xKxafAzy1hFITg</REVIEW_ID>)

> This review is for the delicious breakfast and coffee. The Green Breakfast Bowl and the Biscuit sandwich were both perfectly cooked, fresh, flavorful. We were really impressed with the quality of the cappuccino and the cold brew coffee. The donut was hearty-tasting and clearly a higher quality of bread compared to your typical donut. We came in at around ten on a Saturday and there were plenty of spots to sit even though it is clearly a popular place to be.  (Review ID: <REVIEW_ID>4sGHeBRdn36yOX3KszhzCA</REVIEW_ID>)

> Cold brew was sooo good as were the sandwiches we got. Will most definitely be back. (Review ID: <REVIEW_ID>-6iyNp8Vs9kF0IRGXZvsHA</REVIEW_ID>)

> We got the green eggs and ham, savory scone, cold brew, and mimosas! Their patio is delicious to sit at and enjoy the food! (Review ID: <REVIEW_ID>ybgnv1EGL7U4urhPmPuf4w</REVIEW_ID>)

> Got a breakfast sandwich. Biscuit with egg, bacon and cheese. It comes with greens and some tomato spread, i didn't want that. It was delicious. Cold brew decent also. Cashiers were sweet and accommodating. The girl serving the food had a flat affect. Ambiance is a chill, open, industrial vibe. I wish the outdoor space was larger. There is a wine tasting area in the same spot. Would come here again just for that biscuit! (Review ID: <REVIEW_ID>oFtgQKS4xTrvvgL2eTZKHw</REVIEW_ID>)

> Barista was fast at least. Good latte and cold brew. I should have just ordered a pastry though. (Review ID: <REVIEW_ID>LuLzCVNRkAPjVaRVA__V2A</REVIEW_ID>)

**Caviar & Bananas**

> Need a bottle of wine? Check!
Need a cold brew coffee with just a splash of almond milk? Check!
Need eggs? Check!
Need a gift basket? Check!
Need a frozen Rose? (Oh didn't know I needed that but I DO!) check! 
Neeeeeed that chocolate? Check!
Need a place for an informal meet up. Check! 
Need a fast bar meal 8:00 pm? Check!

 (Review ID: <REVIEW_ID>TO650uj_N9YlOUFjk9K2eQ</REVIEW_ID>)

**Mike's Ice Cream**

> If it's cold, come grab a coffee instead. They offer all the great styles and flavors of coffee as any other coffee house you'll find. (Review ID: <REVIEW_ID>GHAY9_Kpk-1hcq4Ehfmu1w</REVIEW_ID>)

> I am a big fan of diverse ice cream flavors, and this place definitely has that. If you are cold and not necessarily feeling a frosty treat, they also have different hot chocolate flavors from around the world-the Italian hot chocolate is amazing, by the way! The staff were friendly and willing to go the extra mile to ensure that we were enjoying our coffee.  (Review ID: <REVIEW_ID>S2hL5Cnyzs2ck5xiH8WKGQ</REVIEW_ID>)

> Great place to chill with a drink. Process time is short, and tastes are good.  (Review ID: <REVIEW_ID>t77wB9XveTd1jr1SiK-wkw</REVIEW_ID>)

---

**`Source Node 1/10`**

**Node ID:** 37423d12-a5c4-41b4-ba3f-a1d7ec453ae7<br>**Similarity:** -1.7041015625<br>**Text:** Pricey ( a ham and cheese croissant was $5.50)  as with anywhere in Santa Barbra but very delicious. Cute space ( a bit hard to find ). The green eggs and ham  breakfast biscuit is to die for.  ($10 I think) The Santa Barbra tri tip sandwich was excellent! Cold brew coffee was good. Lots of good choices in the grab and go section, and our salami and pesto sammy was also great.<br>**Metadata:** {'review_id': 'Q3fPo_x6xKxafAzy1hFITg', 'user_id': 'ha2Lv7WnxvdYnRsqwL1apQ', 'business_id': 'IDtLPgUrqorrpqSLdfMhZQ', 'review_stars': 4, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2018-07-17T00:20:41', 'biz_name': 'Helena Avenue Bakery', 'biz_address': '131 Anacapa St, Ste C', 'biz_city': 'Santa Barbara', 'biz_state': 'CA', 'biz_postal_code': '93101', 'biz_latitude': 34.4144445, 'biz_longitude': -119.6906718, 'biz_stars': 4.0, 'biz_review_count': 389, 'biz_is_open': 1, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'none'", 'Ambience': "{'touristy': False, 'hipster': True, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': True, 'upscale': False, 'classy': False, 'casual': True}", 'BYOB': None, 'BestNights': None, 'BikeParking': 'True', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': 'True', 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': 'True', 'DriveThru': None, 'GoodForDancing': None, 'GoodForKids': 'True', 'GoodForMeal': "{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': False, 'brunch': True, 'breakfast': True}", 'HappyHour': 'False', 'HasTV': 'False', 'Music': None, 'NoiseLevel': "u'average'", 'OutdoorSeating': 'True', 'RestaurantsAttire': "'casual'", 'RestaurantsDelivery': 'None', 'RestaurantsGoodForGroups': 'True', 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': 'False', 'RestaurantsTableService': 'False', 'RestaurantsTakeOut': 'True', 'Smoking': None, 'WheelchairAccessible': 'True', 'WiFi': "u'no'"}, 'biz_categories': ['Food', 'Restaurants', 'Salad', 'Coffee & Tea', 'Breakfast & Brunch', 'Sandwiches', 'Bakeries'], 'biz_hours': {'Friday': '8:0-14:0', 'Monday': '0:0-0:0', 'Saturday': '8:0-14:0', 'Sunday': '8:0-14:0', 'Thursday': '8:0-14:0', 'Tuesday': '8:0-14:0', 'Wednesday': '8:0-14:0'}}<br>

...

**Node ID:** fb8d1897-1192-4e58-a17f-b53cbac966fa<br>**Similarity:** -4.38671875<br>**Text:** Barista was fast at least. Good latte and cold brew. I should have just ordered a pastry though.<br>**Metadata:** {'review_id': 'LuLzCVNRkAPjVaRVA__V2A', 'user_id': '7gDOm5IJarR2QtOlbY695A', 'business_id': 'IDtLPgUrqorrpqSLdfMhZQ', 'review_stars': 3, 'useful': 2, 'funny': 0, 'cool': 0, 'date': '2018-03-04T17:37:14', 'biz_name': 'Helena Avenue Bakery', 'biz_address': '131 Anacapa St, Ste C', 'biz_city': 'Santa Barbara', 'biz_state': 'CA', 'biz_postal_code': '93101', 'biz_latitude': 34.4144445, 'biz_longitude': -119.6906718, 'biz_stars': 4.0, 'biz_review_count': 389, 'biz_is_open': 1, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'none'", 'Ambience': "{'touristy': False, 'hipster': True, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': True, 'upscale': False, 'classy': False, 'casual': True}", 'BYOB': None, 'BestNights': None, 'BikeParking': 'True', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': 'True', 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': 'True', 'DriveThru': None, 'GoodForDancing': None, 'GoodForKids': 'True', 'GoodForMeal': "{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': False, 'brunch': True, 'breakfast': True}", 'HappyHour': 'False', 'HasTV': 'False', 'Music': None, 'NoiseLevel': "u'average'", 'OutdoorSeating': 'True', 'RestaurantsAttire': "'casual'", 'RestaurantsDelivery': 'None', 'RestaurantsGoodForGroups': 'True', 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': 'False', 'RestaurantsTableService': 'False', 'RestaurantsTakeOut': 'True', 'Smoking': None, 'WheelchairAccessible': 'True', 'WiFi': "u'no'"}, 'biz_categories': ['Food', 'Restaurants', 'Salad', 'Coffee & Tea', 'Breakfast & Brunch', 'Sandwiches', 'Bakeries'], 'biz_hours': {'Friday': '8:0-14:0', 'Monday': '0:0-0:0', 'Saturday': '8:0-14:0', 'Sunday': '8:0-14:0', 'Thursday': '8:0-14:0', 'Tuesday': '8:0-14:0', 'Wednesday': '8:0-14:0'}}<br>

---

**`Source Node 10/10`**

**Node ID:** 5f992b04-edf9-42a7-9cfd-eb01dd622eb4<br>**Similarity:** -5.0546875<br>**Text:** Great place to chill with a drink. Process time is short, and tastes are good.<br>**Metadata:** {'review_id': 't77wB9XveTd1jr1SiK-wkw', 'user_id': 'Ph3YHXzuVLTAibVKAWUMAA', 'business_id': 'oaboaRBUgGjbo2kfUIKDLQ', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2018-02-23T22:33:49', 'biz_name': "Mike's Ice Cream", 'biz_address': '129 2nd Ave N', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37201', 'biz_latitude': 36.1626492, 'biz_longitude': -86.7759733, 'biz_stars': 4.5, 'biz_review_count': 593, 'biz_is_open': 1, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'none'", 'Ambience': "{'romantic': False, 'intimate': False, 'touristy': False, 'hipster': False, 'divey': False, 'classy': False, 'trendy': False, 'upscale': False, 'casual': True}", 'BYOB': None, 'BestNights': None, 'BikeParking': 'False', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}", 'ByAppointmentOnly': 'False', 'Caters': 'False', 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': 'False', 'DriveThru': 'False', 'GoodForDancing': None, 'GoodForKids': 'True', 'GoodForMeal': None, 'HappyHour': None, 'HasTV': 'False', 'Music': None, 'NoiseLevel': "u'average'", 'OutdoorSeating': 'None', 'RestaurantsAttire': "'casual'", 'RestaurantsDelivery': 'None', 'RestaurantsGoodForGroups': 'True', 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': 'False', 'RestaurantsTableService': None, 'RestaurantsTakeOut': 'True', 'Smoking': None, 'WheelchairAccessible': None, 'WiFi': "'free'"}, 'biz_categories': ['Ice Cream & Frozen Yogurt', 'Coffee & Tea', 'Restaurants', 'Sandwiches', 'Food'], 'biz_hours': {'Friday': '8:0-0:0', 'Monday': '8:0-23:0', 'Saturday': '8:0-0:30', 'Sunday': '8:0-23:0', 'Thursday': '8:0-23:0', 'Tuesday': '8:0-23:0', 'Wednesday': '8:0-23:0'}}<br>

{'37423d12-a5c4-41b4-ba3f-a1d7ec453ae7': {'review_id': 'Q3fPo_x6xKxafAzy1hFITg',
  'user_id': 'ha2Lv7WnxvdYnRsqwL1apQ',
  'business_id': 'IDtLPgUrqorrpqSLdfMhZQ',
  'review_stars': 4,
  'useful': 0,
  'funny': 0,
  'cool': 0,
  'date': '2018-07-17T00:20:41',
  'biz_name': 'Helena Avenue Bakery',
  'biz_address': '131 Anacapa St, Ste C',
  'biz_city': 'Santa Barbara',
  'biz_state': 'CA',
  'biz_postal_code': '93101',
  'biz_latitude': 34.4144445,
  'biz_longitude': -119.6906718,
  'biz_stars': 4.0,
  'biz_review_count': 389,
  'biz_is_open': 1,
  'biz_attributes': {'AcceptsInsurance': None,
   'Alcohol': "u'none'",
   'Ambience': "{'touristy': False, 'hipster': True, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': True, 'upscale': False, 'classy': False, 'casual': True}",
   'BYOB': None,
   'BestNights': None,
   'BikeParking': 'True',
   'BusinessAcceptsBitcoin': None,
   'BusinessAcceptsCreditCards': 'True',
   'BusinessParking': "{'garage': False, 'street': True, 'v

## Response Evaluation

In [47]:
from src.run.eval import ResponseEvaluator

In [48]:
response_evaluator = ResponseEvaluator()

### Synthetic

In [49]:
response_eval_documents, response_synthetic_eval_dataset = response_evaluator.generate_synthetic_dataset(cfg, documents)

2024-08-13 21:25:22.702 | INFO     | src.run.eval.response:generate_synthetic_dataset:40 - Sampling 30 documents for response evaluation...
2024-08-13 21:25:22.709 | INFO     | src.run.eval.response:generate_synthetic_dataset:60 - Creating new response eval dataset at data/026_rez_tool/response_synthetic_eval_dataset.json...
2024-08-13 21:25:22.710 | INFO     | src.run.eval.response:generate_synthetic_dataset:63 - Creating synthetic response eval dataset...


Parsing nodes:   0%|          | 0/30 [00:00<?, ?it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.23s/it]
2024-08-13 21:26:20.889 | INFO     | src.run.eval.response:generate_synthetic_dataset:86 - Persisting synthetic response eval dataset at data/026_rez_tool/response_synthetic_eval_dataset.json...


In [50]:
response_synthetic_eval_prediction_dataset = await response_synthetic_eval_dataset.amake_predictions_with(
    predictor=query_engine, batch_size=cfg.batch_size, show_progress=True
)

Batch processing of predictions:   0%|                                                                                                                                                                                                      | 0/1 [00:00<?, ?it/s]

Generated queries:
trendy Japanese restaurants Nashville variety dishes
"I recently dined at a fantastic Japanese restaurant in Nashville that had an amazing selection of dishes, from sushi to ramen. The atmosphere was vibrant and the service was top-notch!"


Batch processing of predictions:   0%|                                                                                                                                                                                                      | 0/1 [00:00<?, ?it/s]

Generated queries:
Italian restaurants excellent customer service Reno NV
"I recently dined at Bella Italia in Reno, and the customer service was outstanding! The staff was attentive and made sure we had everything we needed. Highly recommend for a great dining experience!"


Batch processing of predictions:   0%|                                                                                                                                                                                                      | 0/1 [00:00<?, ?it/s]

...

Batch processing of predictions:   0%|                                                                                                                                                                                                      | 0/1 [00:00<?, ?it/s]

Generated queries:
breweries craft beers indianapolis
"I visited a fantastic brewery in Indianapolis that had an impressive selection of craft beers, including IPAs, stouts, and seasonal brews. The atmosphere was lively, and the staff was knowledgeable about their offerings."


Batch processing of predictions:   0%|                                                                                                                                                                                                      | 0/1 [00:00<?, ?it/s]

Generated queries:
highly-rated wine bars New Orleans
"I visited a fantastic wine bar in New Orleans called Bacchanal, where they have an impressive selection of wines and a lovely outdoor space to enjoy them."


Batch processing of predictions: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:10<00:00, 10.92s/it]


In [51]:
response_synthetic_mean_scores_df, response_synthetic_deep_eval_df = response_evaluator.evaluate_labelled_rag_dataset(
    response_synthetic_eval_dataset,
    response_synthetic_eval_prediction_dataset,
    dataset_name="synthetic",
    judge_model=cfg.eval_cfg.response_eval_llm_model,
    cache_dp=cfg.notebook_cache_dp
)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [04:21<00:00,  8.72s/it]


In [52]:
response_synthetic_mean_scores_df

rag,base_rag
metrics,
mean_correctness_score,4.533333
mean_relevancy_score,0.933333
mean_faithfulness_score,0.966667
mean_context_relevancy_score,0.670833


In [53]:
response_synthetic_deep_eval_df

,query,answer,relevancy_score,correctness_score,faithfulness_score,context_relevancy_score,contexts
0,What are some trendy Japanese restaurants in N...,"\nBased on the provided information, the top r...",1.0,5.0,1.0,0.750,[One of my favorite restaurants in Nashville. ...
1,- What are some Italian restaurants known for ...,\nBased on the provided reviews for Romano's M...,1.0,3.0,1.0,0.625,"[Service was excellent. , Chicken parm was goo..."
2,What are some highly-rated wine bars in New Or...,"\nBased on the provided information, here are ...",1.0,5.0,1.0,0.750,"[Great service, great wine and delicious food...."
3,- What are some highly-rated wine bars in New ...,"\nBased on the provided information, here are ...",1.0,5.0,1.0,0.750,[A great addition NOLA and the CBD. The wine s...
4,What are some ice cream shops that offer uniqu...,\nBased on your query for ice cream shops that...,1.0,5.0,1.0,0.750,[While walking around downtown I kept seeing p...
5,What are some coffeehouses in Edmonton that of...,"\nBased on the provided information, here are ...",1.0,5.0,1.0,0.750,[I went there yesterday afternoon for the firs...
6,What are some highly-rated sports bars that of...,\nBased on your query for highly-rated sports ...,1.0,5.0,1.0,0.625,[Greasy(?) but decent.\n\nThe food menu is not...
7,- What are some cocktail bars in Philadelphia ...,\nBased on your query for cocktail bars in Phi...,1.0,5.0,1.0,0.625,[The best new bar in Philly. Amazing drink men...
8,- What are some sports bars in Indianapolis th...,"\nBased on the provided information, here are ...",1.0,5.0,1.0,0.625,[The best part obviously is the scenery and fr...
9,What are some popular gastropubs in New Orlean...,"\nBased on the provided information, the top r...",1.0,4.0,1.0,0.750,[YUM!!! Copper vine not only has a great atmos...


#### Error Analysis

In [54]:
response_synthetic_eval_dataset_dict = dict()
for example in response_synthetic_eval_dataset.examples:
    response_synthetic_eval_dataset_dict[example.query] = {
        "reference_answer": example.reference_answer,
        "reference_contexts": example.reference_contexts,
    }

In [55]:
response_synthetic_deep_eval_df.sort_values(['relevancy_score', 'correctness_score', 'faithfulness_score', 'context_relevancy_score'])

,query,answer,relevancy_score,correctness_score,faithfulness_score,context_relevancy_score,contexts
20,- What are some sushi bars in Philadelphia tha...,"\nBased on the provided reviews for Tuna Bar, ...",0.0,2.0,0.0,0.625,"[Anyways. Food is great, management sucks. The..."
25,- What are some mobile phone stores known for ...,\nBased on the provided reviews for the T-Mobi...,0.0,3.0,1.0,0.500,[I usually love going to this t-mobile. The re...
1,- What are some Italian restaurants known for ...,\nBased on the provided reviews for Romano's M...,1.0,3.0,1.0,0.625,"[Service was excellent. , Chicken parm was goo..."
10,- What are some popular ice cream shops in Nas...,"\nBased on the provided information, here are ...",1.0,3.0,1.0,0.750,"[Fresh, homemade ice cream! This place is def..."
21,- What are some popular brunch spots in New Or...,"\nBased on the provided information, here are ...",1.0,3.0,1.0,0.750,[Brunch in the Big Easy! There are so many sp...
14,- What are some sports bars in Indianapolis th...,"\nBased on the provided information, here are ...",1.0,4.0,1.0,0.125,[Twin peaks has been open for a couple of mont...
22,- What are some gastropubs in Indianapolis tha...,"\nBased on the provided information, the follo...",1.0,4.0,1.0,0.625,[Def not as good as Chumley's. Probably not co...
9,What are some popular gastropubs in New Orlean...,"\nBased on the provided information, the top r...",1.0,4.0,1.0,0.750,[YUM!!! Copper vine not only has a great atmos...
16,What are some popular coffee shops in Nashvill...,\nBased on your query for popular coffee shops...,1.0,5.0,1.0,0.500,[Caviar & Bananas offers a distinctive food ex...
6,What are some highly-rated sports bars that of...,\nBased on your query for highly-rated sports ...,1.0,5.0,1.0,0.625,[Greasy(?) but decent.\n\nThe food menu is not...


In [56]:
CONTEXT_RELEVANCE_SCORE_THRESHOLD = 0.5

response_synthetic_eval_irrelevance_df = (
    response_synthetic_deep_eval_df
    .loc[lambda df: df['context_relevancy_score'].le(CONTEXT_RELEVANCE_SCORE_THRESHOLD)]
    .sort_values(['context_relevancy_score', 'relevancy_score', 'correctness_score', 'faithfulness_score'])
)

with pd.option_context('display.max_colwidth', 400):
    display(response_synthetic_eval_irrelevance_df)

,query,answer,relevancy_score,correctness_score,faithfulness_score,context_relevancy_score,contexts
14,- What are some sports bars in Indianapolis that offer a better dining experience than Twin Peaks?,"\nBased on the provided information, here are some recommendations for sports bars in Indianapolis that may offer a better dining experience than Twin Peaks, ranked by relevance:\n\n### 1. **Brad Brass Flamingo**\n - **Strengths**: One reviewer explicitly recommended this establishment over Twin Peaks, suggesting that it may provide a more welcoming atmosphere, especially for women. This ind...",1.0,4.0,1.0,0.125,[Twin peaks has been open for a couple of months and I had heard good things about the food and the scenery and decided to check it out. I was told by some of my friends that it was substantially better than Hooters in both regards as well. \n\nWe were seated faster than I expected and initially thought it was strange that we were sitting at a very long table along with others. The table was s...
25,- What are some mobile phone stores known for excellent customer service?,"\nBased on the provided reviews for the T-Mobile store located at 4735 Concord Pike, Wilmington, DE, here are the recommendations ranked by relevance to the user's query about mobile phone stores known for excellent customer service:\n\n### Recommendations:\n\n1. **T-Mobile (4735 Concord Pike, Wilmington, DE)**\n - **Strengths**: \n - Several reviews highlight positive experiences with c...",0.0,3.0,1.0,0.500,"[I usually love going to this t-mobile. The representatives are always so nice and helpful but today, the girl there gave me nothing but attitude when I asked about the new iPhone. , I get awesome service every time I come here. Thing is, they can get pretty crowded, which means long waits., Tried to add a phone to the family plan. The new phone was broken. The worst part is the store said ..."
16,What are some popular coffee shops in Nashville that offer unique food options?,"\nBased on your query for popular coffee shops in Nashville that offer unique food options, here are the top recommendations:\n\n### 1. **Caviar & Bananas**\n - **Address:** 2031 Broadway, Nashville, TN\n - **Review Stars:** 4.5 (average from multiple reviews)\n - **Strengths:**\n - Offers a distinctive urban market experience with a variety of food options including salads, sandwich...",1.0,5.0,1.0,0.500,"[Caviar & Bananas offers a distinctive food experience. i call it a type of urban market. walk into a nice coffee bar, then walk through the beautiful desserts, pass by wines , variety of cheeses, walk pass unique market products, then order a personal specialized salad, or over to the deli market for a custom sandwich and/or lovely freshly made deli selections. watch the bakery at work. ..."


In [57]:
for i, row in response_synthetic_eval_irrelevance_df.reset_index(drop=True).iterrows():
    print(f"\n\n==============Error #{i+1}===============\n\n")
    print(f"Query:\n{row.query}\n")
    contexts = '\n\n'.join(row.contexts)
    print(f"Context:\n{contexts}\n")
    print(f"Answer:\n{row.answer}\n----\n")
    expected = response_synthetic_eval_dataset_dict.get(row.query)
    if not expected:
        logger.error(f"Could not find query {row.query} in synthetic_response_eval_dataset_dict!")
        continue
    expected_answer = expected['reference_answer']
    print(f"Expected Answer:\n{expected_answer}\n")
    expected_contexts = expected['reference_contexts']
    print(f"Expected Contexts:\n{expected_contexts}\n")



==============Error #1===============


Query:
- What are some sports bars in Indianapolis that offer a better dining experience than Twin Peaks?

Context:
Twin peaks has been open for a couple of months and I had heard good things about the food and the scenery and decided to check it out. I was told by some of my friends that it was substantially better than Hooters in both regards as well. 

We were seated faster than I expected and initially thought it was strange that we were sitting at a very long table along with others. The table was separated by a piece of wood with an ax in it. 

Twin Peaks is your typical guys bar, way above Hooters, and on par with tilted kilt.  The thing that puts it over the top is the ICE cold beer served at 29 degrees.

Twin Peaks is your typical guys bar, way above Hooters, and on par with tilted kilt.  The thing that puts it over the top is the ICE cold beer served at 29 degrees.  

If I were to describe twin peaks to someone, I would tell them it i

In [58]:
# query = "- What are some pizza places known for their quality and flavor in White House, TN?"
query = "What are some sports bars in Indianapolis that offer a better dining experience than Twin Peaks?	"
response = query_engine.query(query)
display_response(response, source_length=1000, show_source=True, show_metadata=True, show_source_metadata=True)

Generated queries:
sports bars dining experience indianapolis better than Twin Peaks
"I recently visited a sports bar in Indianapolis that completely surpassed my expectations. The atmosphere was lively, the food was delicious, and the service was top-notch. It was a great alternative to Twin Peaks, offering a diverse menu and a fantastic selection of drinks."


**`Final Response:`** Based on the provided information, here are some recommendations for sports bars in Indianapolis that may offer a better dining experience than Twin Peaks, ranked by relevance:

### 1. **Brad Brass Flamingo**
   - **Strengths**: This establishment is specifically mentioned as a better alternative to Twin Peaks in the negative reviews. While details about the menu and atmosphere are not provided, the mention of it being preferable suggests it may offer a more welcoming environment, especially for women.
   - **Comparison**: Unlike Twin Peaks, which has mixed reviews regarding service and atmosphere, Brad Brass Flamingo seems to be positioned as a more favorable option, particularly for those seeking a more inclusive dining experience.

### 2. **Hooters**
   - **Strengths**: Hooters is frequently compared to Twin Peaks, with some reviews indicating that Twin Peaks is an upscale version of Hooters. Hooters has a well-established reputation and may offer a similar sports bar vibe with a focus on wings and beer.
   - **Comparison**: While Twin Peaks is noted for its cold beer and family-friendly atmosphere, Hooters may provide a more traditional sports bar experience without the hunting lodge theme, which could appeal to those looking for a straightforward dining experience.

### 3. **Tilted Kilt**
   - **Strengths**: Similar to Twin Peaks, Tilted Kilt is known for its themed waitstaff and sports bar atmosphere. It is often mentioned in the same breath as Twin Peaks, suggesting it may offer comparable food and drink options with a different ambiance.
   - **Comparison**: Tilted Kilt may provide a more relaxed environment than Twin Peaks, which has received criticism for its service. The focus on a Celtic theme could also attract those looking for a unique twist on the sports bar experience.

### Summary of Recommendations:
- **Brad Brass Flamingo** stands out as the most recommended alternative, particularly for those who may have felt uncomfortable at Twin Peaks. It is suggested as a better option for women and those seeking a more respectful dining experience.
- **Hooters** and **Tilted Kilt** are both viable options for those who enjoy the themed sports bar concept but may prefer a different atmosphere or menu. They are both well-known chains that offer a similar experience to Twin Peaks but with varying degrees of service and ambiance.

In conclusion, if the user is looking for a sports bar in Indianapolis that offers a better dining experience than Twin Peaks, **Brad Brass Flamingo** is the top recommendation, followed by **Hooters** and **Tilted Kilt** for those who appreciate themed dining but want to explore alternatives.


#### Referenced Paragraphs
**Twin Peaks**

> Twin peaks has been open for a couple of months and I had heard good things about the food and the scenery and decided to check it out. I was told by some of my friends that it was substantially better than Hooters in both regards as well. 

We were seated faster than I expected and initially thought it was strange that we were sitting at a very long table along with others. The table was separated by a piece of wood with an ax in it.  (Review ID: <REVIEW_ID>l3jFrMXk3h99WlTv-q9gNw</REVIEW_ID>)

> Twin Peaks is your typical guys bar, way above Hooters, and on par with tilted kilt.  The thing that puts it over the top is the ICE cold beer served at 29 degrees.   (Review ID: <REVIEW_ID>KsmnyFmghtOx_IbOG9RINg</REVIEW_ID>)

> Twin Peaks is your typical guys bar, way above Hooters, and on par with tilted kilt.  The thing that puts it over the top is the ICE cold beer served at 29 degrees. (Review ID: <REVIEW_ID>jAvNoTtB1x2LELl_-7y8kg</REVIEW_ID>)

> If I were to describe twin peaks to someone, I would tell them it is an upscale Hooters.  The main difference between Hooters and twin peaks is the quality and the customer service. I'm not saying hooters is bad and does not have good service, but I would pick twin peaks over hooters any day of the week.    (Review ID: <REVIEW_ID>1Tn6LTyYfEwC0TSPsxohXw</REVIEW_ID>)

> ...Her service was over-the-top, too  (not once did I go lacking for anything), but two other things stood out in my mind as well.  While Twin Peaks is a sports-bar/restaurant, they're very family friendly.  That's unusual for many sports bars/restaurants I've visited. Singles, couples, families, and sports enthusiasts waiting to watch the games were all present. And I can't begin to describe how delicious the food was!  For lunch I had "Mom's Pot Roast," but to be honest mom never made it this great.  My dessert was a Twin Peaks Sundae that was amazing.  For my return trip at dinner, I specifically asked if Arielle was available.  I was informed I was in luck.  She had worked a longer shift than usual today.  I wanted to eat lighter, so I had a cup of Blonde Bean with Bacon Soup (delicious!) and asked Arielle specific questions about the items on the dessert menu I still hadn't tried.  (Again, there was no impatience or indifference.  She was warm and friendly and seemed eager to assist ...

> ...If you happen to stumble into Twin Peaks expecting an otherworldly encounter with David Lynch, you're probably going to be disappointed. 

On the other hand, if you stumble into the brand new Indianapolis location of the Texas-based chain of "breastaurants" called Twin Peaks hoping to enjoy some good eats, good drinks, and scenic views, then you'll likely be incredibly happy. 

Founded in 2005 in Lewisville, Texas, Twin Peaks opened for the dinner hour starting on 12/8/14 at the former Loon Lake Lodge just East of Castleton at the corner of I-69 and East 82nd Street. 

With its servers all wearing the trademark Twin Peaks cleavage and midriff-baring red plaid tops and khaki shorts, the concept behind Twin Peaks may sound familiar but this variation on the familiar theme is far less kitschy and far more personable than other similarly themed restaurants.

The easiest way to describe Twin Peaks is sports bar meets hunting lodge meets bevy of beautiful women. For those who remember Loon L...

> Do not recommend for women to go there. Strippers will even treat people better than these chicks do. Go to brad brass flamingo before twin peaks (Review ID: <REVIEW_ID>B5MlUutuPW6F8G7NAvGJhg</REVIEW_ID>)

> I'll NEVER go back to this dump. I'll never recommend this dump. Give me a good experience in a dump and and I'm good. A bad experience in a dump ? NO WAY. 

Get your reality readjusted, Twin Peaks. There are better places for food etc. unless you rate food according to breast exposure. I'll be back in Indy fir business but I'll never be back here. (Review ID: <REVIEW_ID>vne8OJzpDAQ6SmmRQG7nFQ</REVIEW_ID>)

> I'm a big sports bar guy and a sports bar food guy and don't expect a sports bar to be 5 star dining. Let me answer a question that people ask me when I said I ate there. "Is it okay to bring your wife and kids there to watch a game?" The servers do wear low cut shirts and shorts, but nothing that was any different than other themed sports bars. When I was there, there were groups of women together and also couples there.  (Review ID: <REVIEW_ID>grTir4NbamCVuxnzfa8sqw</REVIEW_ID>)

> We stop by at Twin Peaks on our way to Indianapolis to watch the U of L and Purdue game. This was a cool place. It is a sports bar  that had waitresses that are dressed in Specialty little outfits kind of like a checkered shirt with shorts on it kind of fit the outdoor theme of it it was very like Cabela's meets Hooters is the best way I can put it at Giant TVs everywhere the bathrooms were phenomenal they had like TVs right at each year and I really enjoyed that bathroom so huge! And I mean huge just when you thought that a bathroom couldn't be that large it is.  (Review ID: <REVIEW_ID>4U3_8fv3RtU_lif2Ut5ESg</REVIEW_ID>)

---

**`Source Node 1/10`**

**Node ID:** d438e4a0-0687-40dd-a7c9-aeb90a190607<br>**Similarity:** 0.8349609375<br>**Text:** Twin peaks has been open for a couple of months and I had heard good things about the food and the scenery and decided to check it out. I was told by some of my friends that it was substantially better than Hooters in both regards as well. 

We were seated faster than I expected and initially thought it was strange that we were sitting at a very long table along with others. The table was separated by a piece of wood with an ax in it.<br>**Metadata:** {'review_id': 'l3jFrMXk3h99WlTv-q9gNw', 'user_id': '-0YrXUvXz8112yHap35V2g', 'business_id': 'seKihQKpGGnCeLuELRQPSQ', 'review_stars': 3, 'useful': 3, 'funny': 1, 'cool': 0, 'date': '2015-01-31T03:20:27', 'biz_name': 'Twin Peaks', 'biz_address': '6880 E 82nd St', 'biz_city': 'Indianapolis', 'biz_state': 'IN', 'biz_postal_code': '46250', 'biz_latitude': 39.9062952, 'biz_longitude': -86.0474634, 'biz_stars': 3.5, 'biz_review_count': 257, 'biz_is_open': 1, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "'full_bar'", 'Ambience': "{u'divey': False, u'hipster': None, u'casual': True, u'touristy': None, u'trendy': None, u'intimate': False, u'romantic': False, u'classy': False, u'upscale': False}", 'BYOB': None, 'BestNights': "{u'monday': False, u'tuesday': True, u'wednesday': False, u'thursday': False, u'friday': True, u'saturday': True, u'sunday': False}", 'BikeParking': 'False', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': 'False', 'CoatCheck': 'False', 'Corkage': None, 'DogsAllowed': 'False', 'DriveThru': None, 'GoodForDancing': 'False', 'GoodForKids': 'True', 'GoodForMeal': "{'dessert': None, 'latenight': None, 'lunch': True, 'dinner': True, 'brunch': False, 'breakfast': False}", 'HappyHour': 'True', 'HasTV': 'True', 'Music': "{'dj': False}", 'NoiseLevel': "'average'", 'OutdoorSeating': 'True', 'RestaurantsAttire': "'casual'", 'RestaurantsDelivery': 'False', 'RestaurantsGoodForGroups': 'True', 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': 'False', 'RestaurantsTableService': 'True', 'RestaurantsTakeOut': 'True', 'Smoking': "u'no'", 'WheelchairAccessible': None, 'WiFi': "'free'"}, 'biz_categories': ['Sports Bars', 'American (New)', 'American (Traditional)', 'Nightlife', 'Bars', 'Restaurants'], 'biz_hours': {'Friday': '11:0-19:0', 'Monday': '0:0-0:0', 'Saturday': '11:0-1:0', 'Sunday': '11:0-0:0', 'Thursday': '11:0-22:0', 'Tuesday': '11:0-0:0', 'Wednesday': '11:0-0:0'}}<br>

...

**Node ID:** 6157ede7-619a-4925-86f1-50e4516fc115<br>**Similarity:** -0.5419921875<br>**Text:** I'm a big sports bar guy and a sports bar food guy and don't expect a sports bar to be 5 star dining. Let me answer a question that people ask me when I said I ate there. "Is it okay to bring your wife and kids there to watch a game?" The servers do wear low cut shirts and shorts, but nothing that was any different than other themed sports bars. When I was there, there were groups of women together and also couples there.<br>**Metadata:** {'review_id': 'grTir4NbamCVuxnzfa8sqw', 'user_id': 'sZh1Pp958CUE5Tv3y5xtAQ', 'business_id': 'seKihQKpGGnCeLuELRQPSQ', 'review_stars': 4, 'useful': 3, 'funny': 1, 'cool': 2, 'date': '2015-11-04T16:52:57', 'biz_name': 'Twin Peaks', 'biz_address': '6880 E 82nd St', 'biz_city': 'Indianapolis', 'biz_state': 'IN', 'biz_postal_code': '46250', 'biz_latitude': 39.9062952, 'biz_longitude': -86.0474634, 'biz_stars': 3.5, 'biz_review_count': 257, 'biz_is_open': 1, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "'full_bar'", 'Ambience': "{u'divey': False, u'hipster': None, u'casual': True, u'touristy': None, u'trendy': None, u'intimate': False, u'romantic': False, u'classy': False, u'upscale': False}", 'BYOB': None, 'BestNights': "{u'monday': False, u'tuesday': True, u'wednesday': False, u'thursday': False, u'friday': True, u'saturday': True, u'sunday': False}", 'BikeParking': 'False', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': 'False', 'CoatCheck': 'False', 'Corkage': None, 'DogsAllowed': 'False', 'DriveThru': None, 'GoodForDancing': 'False', 'GoodForKids': 'True', 'GoodForMeal': "{'dessert': None, 'latenight': None, 'lunch': True, 'dinner': True, 'brunch': False, 'breakfast': False}", 'HappyHour': 'True', 'HasTV': 'True', 'Music': "{'dj': False}", 'NoiseLevel': "'average'", 'OutdoorSeating': 'True', 'RestaurantsAttire': "'casual'", 'RestaurantsDelivery': 'False', 'RestaurantsGoodForGroups': 'True', 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': 'False', 'RestaurantsTableService': 'True', 'RestaurantsTakeOut': 'True', 'Smoking': "u'no'", 'WheelchairAccessible': None, 'WiFi': "'free'"}, 'biz_categories': ['Sports Bars', 'American (New)', 'American (Traditional)', 'Nightlife', 'Bars', 'Restaurants'], 'biz_hours': {'Friday': '11:0-19:0', 'Monday': '0:0-0:0', 'Saturday': '11:0-1:0', 'Sunday': '11:0-0:0', 'Thursday': '11:0-22:0', 'Tuesday': '11:0-0:0', 'Wednesday': '11:0-0:0'}}<br>

---

**`Source Node 10/10`**

**Node ID:** fc4ea25f-b6e0-401a-98af-30091fe68c6f<br>**Similarity:** -0.59765625<br>**Text:** We stop by at Twin Peaks on our way to Indianapolis to watch the U of L and Purdue game. This was a cool place. It is a sports bar  that had waitresses that are dressed in Specialty little outfits kind of like a checkered shirt with shorts on it kind of fit the outdoor theme of it it was very like Cabela's meets Hooters is the best way I can put it at Giant TVs everywhere the bathrooms were phenomenal they had like TVs right at each year and I really enjoyed that bathroom so huge! And I mean huge just when you thought that a bathroom couldn't be that large it is.<br>**Metadata:** {'review_id': '4U3_8fv3RtU_lif2Ut5ESg', 'user_id': 'PlkcpVx74F5bcjTznVoTEg', 'business_id': 'seKihQKpGGnCeLuELRQPSQ', 'review_stars': 4, 'useful': 1, 'funny': 0, 'cool': 1, 'date': '2017-09-03T20:07:59', 'biz_name': 'Twin Peaks', 'biz_address': '6880 E 82nd St', 'biz_city': 'Indianapolis', 'biz_state': 'IN', 'biz_postal_code': '46250', 'biz_latitude': 39.9062952, 'biz_longitude': -86.0474634, 'biz_stars': 3.5, 'biz_review_count': 257, 'biz_is_open': 1, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "'full_bar'", 'Ambience': "{u'divey': False, u'hipster': None, u'casual': True, u'touristy': None, u'trendy': None, u'intimate': False, u'romantic': False, u'classy': False, u'upscale': False}", 'BYOB': None, 'BestNights': "{u'monday': False, u'tuesday': True, u'wednesday': False, u'thursday': False, u'friday': True, u'saturday': True, u'sunday': False}", 'BikeParking': 'False', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': 'False', 'CoatCheck': 'False', 'Corkage': None, 'DogsAllowed': 'False', 'DriveThru': None, 'GoodForDancing': 'False', 'GoodForKids': 'True', 'GoodForMeal': "{'dessert': None, 'latenight': None, 'lunch': True, 'dinner': True, 'brunch': False, 'breakfast': False}", 'HappyHour': 'True', 'HasTV': 'True', 'Music': "{'dj': False}", 'NoiseLevel': "'average'", 'OutdoorSeating': 'True', 'RestaurantsAttire': "'casual'", 'RestaurantsDelivery': 'False', 'RestaurantsGoodForGroups': 'True', 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': 'False', 'RestaurantsTableService': 'True', 'RestaurantsTakeOut': 'True', 'Smoking': "u'no'", 'WheelchairAccessible': None, 'WiFi': "'free'"}, 'biz_categories': ['Sports Bars', 'American (New)', 'American (Traditional)', 'Nightlife', 'Bars', 'Restaurants'], 'biz_hours': {'Friday': '11:0-19:0', 'Monday': '0:0-0:0', 'Saturday': '11:0-1:0', 'Sunday': '11:0-0:0', 'Thursday': '11:0-22:0', 'Tuesday': '11:0-0:0', 'Wednesday': '11:0-0:0'}}<br>

{'d438e4a0-0687-40dd-a7c9-aeb90a190607': {'review_id': 'l3jFrMXk3h99WlTv-q9gNw',
  'user_id': '-0YrXUvXz8112yHap35V2g',
  'business_id': 'seKihQKpGGnCeLuELRQPSQ',
  'review_stars': 3,
  'useful': 3,
  'funny': 1,
  'cool': 0,
  'date': '2015-01-31T03:20:27',
  'biz_name': 'Twin Peaks',
  'biz_address': '6880 E 82nd St',
  'biz_city': 'Indianapolis',
  'biz_state': 'IN',
  'biz_postal_code': '46250',
  'biz_latitude': 39.9062952,
  'biz_longitude': -86.0474634,
  'biz_stars': 3.5,
  'biz_review_count': 257,
  'biz_is_open': 1,
  'biz_attributes': {'AcceptsInsurance': None,
   'Alcohol': "'full_bar'",
   'Ambience': "{u'divey': False, u'hipster': None, u'casual': True, u'touristy': None, u'trendy': None, u'intimate': False, u'romantic': False, u'classy': False, u'upscale': False}",
   'BYOB': None,
   'BestNights': "{u'monday': False, u'tuesday': True, u'wednesday': False, u'thursday': False, u'friday': True, u'saturday': True, u'sunday': False}",
   'BikeParking': 'False',
   'BusinessA

### Manually Curated
Ref: https://docs.llamaindex.ai/en/stable/examples/llama_dataset/ragdataset_submission_template/#1c-creating-a-labelledragdataset-from-scratch-with-manually-constructed-examples

In [59]:
response_curated_eval_dataset = response_evaluator.generate_curated_dataset(cfg)

2024-08-13 21:36:39.945 | INFO     | src.run.eval.response:generate_curated_dataset:125 - Persisting curated response eval dataset at data/026_rez_tool/response_curated_eval_dataset.json...


In [60]:
response_curated_eval_prediction_dataset = await response_curated_eval_dataset.amake_predictions_with(
    predictor=query_engine, batch_size=cfg.batch_size, show_progress=True
)

Batch processing of predictions:   0%|                                                                                                                                                                                                      | 0/1 [00:00<?, ?it/s]

Generated queries:
recommended restaurants Santa Barbara
"I recently dined at The Lark in Santa Barbara, and the atmosphere was fantastic! The menu features locally sourced ingredients, and I highly recommend their seasonal dishes. It's a must-visit for anyone looking for a great meal nearby."


Batch processing of predictions:   0%|                                                                                                                                                                                                      | 0/1 [00:00<?, ?it/s]

Generated queries:
sports bars dining experience indianapolis better than twin peaks
"I visited a sports bar in Indianapolis that had an amazing atmosphere and delicious food, far surpassing my experience at Twin Peaks. The menu was diverse, and the service was top-notch!"


Batch processing of predictions: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:14<00:00, 14.41s/it]


In [61]:
response_curated_mean_scores_df, response_curated_deep_eval_df = response_evaluator.evaluate_labelled_rag_dataset(
    response_curated_eval_dataset,
    response_curated_eval_prediction_dataset,
    dataset_name="curated",
    judge_model=cfg.eval_cfg.response_eval_llm_model,
    cache_dp=cfg.notebook_cache_dp
)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:19<00:00,  9.83s/it]


In [62]:
response_curated_mean_scores_df

rag,base_rag
metrics,
mean_correctness_score,3.5
mean_relevancy_score,1.0
mean_faithfulness_score,1.0
mean_context_relevancy_score,0.5


In [63]:
response_curated_deep_eval_df

,query,answer,relevancy_score,correctness_score,faithfulness_score,context_relevancy_score,contexts
0,What are the recommended restaurants nearby Sa...,"\nBased on the provided information, the recom...",1.0,4.0,1.0,0.75,[Finally some good food in downtown Santa Barb...
1,What are some sports bars in Indianapolis that...,"\nBased on the provided information, here are ...",1.0,3.0,1.0,0.25,[Twin peaks has been open for a couple of mont...


#### Answers for target questions

In [64]:
from IPython.display import Markdown

In [65]:
from src.run.eval.manual_eval_dataset import MANUAL_EVAL_QA

In [66]:
for question, expected_answer in MANUAL_EVAL_QA:
    display(Markdown(f"\n\n### Question: {question}\n"))
    response = query_engine.query(question)
    display_response(response)



### Question: What are the recommended restaurants nearby Santa Barbara?


Generated queries:
recommended restaurants Santa Barbara
"I recently dined at a fantastic Italian restaurant near Santa Barbara called Trattoria Vittoria. The pasta was homemade and the ambiance was cozy. Highly recommend it for a nice evening out!"


**`Final Response:`** Based on the provided information, the top recommendation for restaurants in Santa Barbara is:

### 1. Helena Avenue Bakery
- **Address:** 131 Anacapa St, Ste C, Santa Barbara, CA
- **Categories:** Food, Restaurants, Salad, Coffee & Tea, Breakfast & Brunch, Sandwiches, Bakeries
- **Rating:** 5 stars (with some mixed reviews)

**Strengths:**
- **Variety of Options:** Helena Avenue Bakery offers a diverse menu that includes breakfast, brunch, sandwiches, salads, and baked goods, making it suitable for various meal times and preferences.
- **Quality of Food:** The reviews highlight the quality of the food, with mentions of delicious items like chicken sandwiches, jams, scones, and gluten-free options.
- **Atmosphere:** The bakery has a lovely ambiance with outdoor seating, which is appreciated by visitors, especially those looking to enjoy a meal in a pleasant environment.
- **Location:** It is conveniently located within walking distance to the beach, making it a great stop for tourists exploring Santa Barbara.

**Comparison:**
- **For Tourists:** Helena Avenue Bakery is highly recommended for tourists due to its quality food and convenient location near the beach. The positive reviews emphasize it as a must-visit spot for those spending the weekend in Santa Barbara.
- **For Locals:** While some reviews suggest it may not be the top choice for locals due to the presence of other brunch spots, the bakery still garners high praise for its food quality and service, particularly for those with dietary restrictions.

### Summary
Helena Avenue Bakery stands out as the primary recommendation for restaurants in Santa Barbara due to its excellent food, variety, and inviting atmosphere. While there are mixed reviews regarding service, the overall experience seems to be positive, especially for visitors. If you're looking for a place to enjoy a meal with a local vibe and good food, this bakery is a solid choice.


#### Referenced Paragraphs
**Helena Avenue Bakery**

> Finally some good food in downtown Santa Barbara.  Everything on the menu is great and could not recommend more.  (Review ID: <REVIEW_ID>8qeD6EhCxUEBj1dO4J4KfA</REVIEW_ID>)

> If you're a tourist and spending the weekend in Santa Barbara, I'd recommend to give this place a go for sure. If you're a local, then not so much.  (Review ID: <REVIEW_ID>Mqh2anPpP_vvLtqCAk_1Xw</REVIEW_ID>)

> Another gem in Santa Barbara! Love the location, walking distance to the beach.  (Review ID: <REVIEW_ID>_S_Lnze72utd6e95FSnw6A</REVIEW_ID>)

> They offered outdoor seating which they disinfected after each attendee, and their hot chocolate was yummy. Definitely would recommend them, especially if you're in Santa Barbara and looking for a gluten-free breakfast at a cafe. (Review ID: <REVIEW_ID>FwB2dNaeap2oORCc88ubUQ</REVIEW_ID>)

> We took a 5 minute walk to the beach boardwalk afterwards. I recommend stopping by! (Review ID: <REVIEW_ID>XwT8iMkd4X548CD_tw99Qg</REVIEW_ID>)

> the service was okay; we had to clean our table ourselves as no one came to clean it. i don't think i'll be returning again given there are better brunch places around downtown santa barbara. (Review ID: <REVIEW_ID>xBgLhU4JvWCL6oHWxXQwjg</REVIEW_ID>)

> Lovely little coffee/breakfast/brunch spot on State Street in Santa Barbara. We went to Helena twice on our trip.  (Review ID: <REVIEW_ID>kZVsvf04NRwYI4X3RZtIiQ</REVIEW_ID>)

> The chicken sandwich was buttery at the same time it was peppery, and sweet while it was salty. For breakfast on the go, this has got to be one of the best places in Santa Barbara. (Review ID: <REVIEW_ID>OwspRoiswk7AzSIdFK-UDw</REVIEW_ID>)

> O....M....G!!!! I'm so glad this place is in Santa Barbara and not my hometown, because I would never stop eating. Their jams, scones, bagels, breads...delish!!!  (Review ID: <REVIEW_ID>8uAtrjUhdq5nFeHXllueBA</REVIEW_ID>)

> First time in Santa Barbara and we came upon this cute spot! Our family has a lot of dietary restrictions and our server was so knowledgable and helpful guiding us through the menu. Highly recommend!  (Review ID: <REVIEW_ID>UcTKZYl3_wydLoQCSx4Rwg</REVIEW_ID>)



### Question: What are some sports bars in Indianapolis that offer a better dining experience than Twin Peaks?


Generated queries:
sports bars dining experience indianapolis better than Twin Peaks
"I visited a sports bar in Indianapolis called The District Tap, and I was impressed by their extensive menu and great atmosphere. The food was delicious, and it felt much more upscale compared to Twin Peaks."


**`Final Response:`** Based on the provided information, here are some recommendations for sports bars in Indianapolis that may offer a better dining experience than Twin Peaks, ranked by relevance:

### 1. **Brad Brass Flamingo**
   - **Strengths**: Although specific details about this venue are not provided, it is mentioned in a negative review of Twin Peaks as a preferable alternative. This suggests that it may offer a more welcoming atmosphere and better service, particularly for women.
   - **Comparison**: Unlike Twin Peaks, which has mixed reviews regarding service and atmosphere, Brad Brass Flamingo is implied to provide a more positive experience, especially for female patrons.

### 2. **Hooters**
   - **Strengths**: Hooters is mentioned as a direct comparison to Twin Peaks, with some reviews indicating that while it is a similar concept, it may not match the quality and service of Twin Peaks. However, it is still a well-known sports bar chain with a consistent dining experience.
   - **Comparison**: While Twin Peaks is described as an upscale version of Hooters, some patrons may prefer the familiarity and established reputation of Hooters, especially if they are looking for a more traditional sports bar experience.

### 3. **Tilted Kilt**
   - **Strengths**: Similar to Twin Peaks, Tilted Kilt is another themed sports bar that is often compared to Hooters. It is noted for its cold beer and a lively atmosphere, which may appeal to sports fans.
   - **Comparison**: Tilted Kilt is positioned as a competitor to Twin Peaks, and while it may offer a similar experience, it could provide a different ambiance and menu options that some patrons might prefer.

### Summary of Recommendations:
- **Brad Brass Flamingo** stands out as a recommended alternative based on the negative feedback about Twin Peaks, suggesting it may offer a more positive dining experience, especially for women.
- **Hooters** and **Tilted Kilt** are both established sports bar options that provide a similar atmosphere to Twin Peaks but may offer different service levels and menu items. They are worth considering for those who enjoy themed sports bars but are looking for a different experience than what Twin Peaks offers.

### Conclusion:
If the user is seeking a better dining experience than Twin Peaks, **Brad Brass Flamingo** is the top recommendation based on implied quality. **Hooters** and **Tilted Kilt** are also viable options for those who enjoy the themed sports bar concept but may want to explore different atmospheres and service styles.


#### Referenced Paragraphs
**Twin Peaks**

> Twin peaks has been open for a couple of months and I had heard good things about the food and the scenery and decided to check it out. I was told by some of my friends that it was substantially better than Hooters in both regards as well. 

We were seated faster than I expected and initially thought it was strange that we were sitting at a very long table along with others. The table was separated by a piece of wood with an ax in it.  (Review ID: <REVIEW_ID>l3jFrMXk3h99WlTv-q9gNw</REVIEW_ID>)

> Twin Peaks is your typical guys bar, way above Hooters, and on par with tilted kilt.  The thing that puts it over the top is the ICE cold beer served at 29 degrees.   (Review ID: <REVIEW_ID>KsmnyFmghtOx_IbOG9RINg</REVIEW_ID>)

> Twin Peaks is your typical guys bar, way above Hooters, and on par with tilted kilt.  The thing that puts it over the top is the ICE cold beer served at 29 degrees. (Review ID: <REVIEW_ID>jAvNoTtB1x2LELl_-7y8kg</REVIEW_ID>)

> If I were to describe twin peaks to someone, I would tell them it is an upscale Hooters.  The main difference between Hooters and twin peaks is the quality and the customer service. I'm not saying hooters is bad and does not have good service, but I would pick twin peaks over hooters any day of the week.    (Review ID: <REVIEW_ID>1Tn6LTyYfEwC0TSPsxohXw</REVIEW_ID>)

> ...Her service was over-the-top, too  (not once did I go lacking for anything), but two other things stood out in my mind as well.  While Twin Peaks is a sports-bar/restaurant, they're very family friendly.  That's unusual for many sports bars/restaurants I've visited. Singles, couples, families, and sports enthusiasts waiting to watch the games were all present. And I can't begin to describe how delicious the food was!  For lunch I had "Mom's Pot Roast," but to be honest mom never made it this great.  My dessert was a Twin Peaks Sundae that was amazing.  For my return trip at dinner, I specifically asked if Arielle was available.  I was informed I was in luck.  She had worked a longer shift than usual today.  I wanted to eat lighter, so I had a cup of Blonde Bean with Bacon Soup (delicious!) and asked Arielle specific questions about the items on the dessert menu I still hadn't tried.  (Again, there was no impatience or indifference.  She was warm and friendly and seemed eager to assist ...

> ...If you happen to stumble into Twin Peaks expecting an otherworldly encounter with David Lynch, you're probably going to be disappointed. 

On the other hand, if you stumble into the brand new Indianapolis location of the Texas-based chain of "breastaurants" called Twin Peaks hoping to enjoy some good eats, good drinks, and scenic views, then you'll likely be incredibly happy. 

Founded in 2005 in Lewisville, Texas, Twin Peaks opened for the dinner hour starting on 12/8/14 at the former Loon Lake Lodge just East of Castleton at the corner of I-69 and East 82nd Street. 

With its servers all wearing the trademark Twin Peaks cleavage and midriff-baring red plaid tops and khaki shorts, the concept behind Twin Peaks may sound familiar but this variation on the familiar theme is far less kitschy and far more personable than other similarly themed restaurants.

The easiest way to describe Twin Peaks is sports bar meets hunting lodge meets bevy of beautiful women. For those who remember Loon L...

> Do not recommend for women to go there. Strippers will even treat people better than these chicks do. Go to brad brass flamingo before twin peaks (Review ID: <REVIEW_ID>B5MlUutuPW6F8G7NAvGJhg</REVIEW_ID>)

> I'll NEVER go back to this dump. I'll never recommend this dump. Give me a good experience in a dump and and I'm good. A bad experience in a dump ? NO WAY. 

Get your reality readjusted, Twin Peaks. There are better places for food etc. unless you rate food according to breast exposure. I'll be back in Indy fir business but I'll never be back here. (Review ID: <REVIEW_ID>vne8OJzpDAQ6SmmRQG7nFQ</REVIEW_ID>)

> I'm a big sports bar guy and a sports bar food guy and don't expect a sports bar to be 5 star dining. Let me answer a question that people ask me when I said I ate there. "Is it okay to bring your wife and kids there to watch a game?" The servers do wear low cut shirts and shorts, but nothing that was any different than other themed sports bars. When I was there, there were groups of women together and also couples there.  (Review ID: <REVIEW_ID>grTir4NbamCVuxnzfa8sqw</REVIEW_ID>)

> We stop by at Twin Peaks on our way to Indianapolis to watch the U of L and Purdue game. This was a cool place. It is a sports bar  that had waitresses that are dressed in Specialty little outfits kind of like a checkered shirt with shorts on it kind of fit the outdoor theme of it it was very like Cabela's meets Hooters is the best way I can put it at Giant TVs everywhere the bathrooms were phenomenal they had like TVs right at each year and I really enjoyed that bathroom so huge! And I mean huge just when you thought that a bathroom couldn't be that large it is.  (Review ID: <REVIEW_ID>4U3_8fv3RtU_lif2Ut5ESg</REVIEW_ID>)

### Error Analysis

In [67]:
response_curated_eval_dataset_dict = dict()
for example in response_curated_eval_dataset.examples:
    response_curated_eval_dataset_dict[example.query] = {
        "reference_answer": example.reference_answer,
        "reference_contexts": example.reference_contexts,
    }

In [68]:
response_curated_deep_eval_df.sort_values(['relevancy_score', 'correctness_score', 'faithfulness_score'])

,query,answer,relevancy_score,correctness_score,faithfulness_score,context_relevancy_score,contexts
1,What are some sports bars in Indianapolis that...,"\nBased on the provided information, here are ...",1.0,3.0,1.0,0.25,[Twin peaks has been open for a couple of mont...
0,What are the recommended restaurants nearby Sa...,"\nBased on the provided information, the recom...",1.0,4.0,1.0,0.75,[Finally some good food in downtown Santa Barb...


In [69]:
response_curated_eval_irrelevance_df = (
    response_curated_deep_eval_df
    .loc[lambda df: df['relevancy_score'].lt(1)]
    .sort_values(['relevancy_score', 'correctness_score', 'faithfulness_score'])
)

for i, row in response_curated_eval_irrelevance_df.reset_index(drop=True).iterrows():
    print(f"\n\n==============Error #{i+1}===============\n\n")
    print(f"Query:\n{row.query}\n")
    contexts = '\n\n'.join(row.contexts)
    print(f"Context:\n{contexts}\n")
    print(f"Answer:\n{row.answer}\n----\n")
    expected = response_curated_eval_dataset_dict.get(row.query)
    if not expected:
        logger.error(f"Could not find query {row.query} in synthetic_response_eval_dataset_dict!")
        continue
    expected_answer = expected['reference_answer']
    print(f"Expected Answer:\n{expected_answer}\n")
    expected_contexts = expected['reference_contexts']
    print(f"Expected Contexts:\n{expected_contexts}\n")

# Chat Bot Quick Test

In [85]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

from src.svc.availability.availability_check import ReservationService

In [86]:
query_engine_tool = QueryEngineTool(
    query_engine=query_engine,
    metadata=ToolMetadata(
        name="reco_review",
        description=(
            "useful for when you want to find places to visit"
            " based on end-user reviews"
        ),
    ),
)

opening_hours_db = data.set_index('business_id')['biz_hours'].dropna().to_dict()
biz_name_id_mapper = data.set_index('biz_name')['business_id'].dropna().to_dict()
rez_tool = ReservationService(opening_hours_db, biz_name_id_mapper)

tools = [query_engine_tool] + rez_tool.to_tool_list()

In [87]:
from llama_index.agent.openai import OpenAIAgent

agent = OpenAIAgent.from_tools(tools, verbose=True)

In [88]:
response = agent.chat("hi, i am quy")
print(str(response))

response = agent.chat(
    "Where to find some good cold brew coffee?"
)
print(str(response))

response = agent.chat(
    "where is it?"
)
print(str(response))

response = agent.chat(
    "what is the best time to visit?"
)
print(str(response))

Added user message to memory: hi, i am quy
Hello Quy! How can I assist you today?
Added user message to memory: Where to find some good cold brew coffee?
=== Calling Function ===
Calling function: reco_review with args: {"input":"cold brew coffee"}
Generated queries:
cold brew coffee
"I recently tried a new café that specializes in cold brew coffee, and I was amazed by the smooth flavor and rich aroma. They also offer various blends and seasonal flavors that are perfect for any coffee lover."
Got output: 
Based on your interest in cold brew coffee, here are several recommendations ranked by relevance:

### 1. **Helena Avenue Bakery**
   - **Location:** 131 Anacapa St, Ste C, Santa Barbara, CA
   - **Rating:** 5 stars
   - **Strengths:** Helena Avenue Bakery is highly praised for its cold brew coffee, which has received multiple mentions in reviews for being exceptionally good. The bakery also offers a variety of delicious breakfast options, including sandwiches and pastries, making it 

In [89]:
response = agent.chat(
    "Can I book for 5 pax on August 15 at 8 AM?"
)
print(str(response))

Added user message to memory: Can I book for 5 pax on August 15 at 8 AM?
=== Calling Function ===
Calling function: get_business_id with args: {"biz_name": "Helena Avenue Bakery"}
Got output: IDtLPgUrqorrpqSLdfMhZQ

=== Calling Function ===
Calling function: get_business_id with args: {"biz_name": "Caviar & Bananas"}
Got output: lk9IwjZXqUMqqOhM774DtQ

=== Calling Function ===
Calling function: get_business_id with args: {"biz_name": "Wawa"}
Got output: ppFCk9aQkM338Rgwpl2F5A

=== Calling Function ===
Calling function: get_business_id with args: {"biz_name": "Barnes & Noble Booksellers"}
Got output: qhDdDeI3K4jy2KyzwFN53w

=== Calling Function ===
Calling function: check_availability with args: {"business_id": "IDtLPgUrqorrpqSLdfMhZQ", "date_time": "2023-08-15T08:00:00", "pax": 5}
Got output: True

=== Calling Function ===
Calling function: check_availability with args: {"business_id": "lk9IwjZXqUMqqOhM774DtQ", "date_time": "2023-08-15T08:00:00", "pax": 5}
Got output: False

=== Callin

In [77]:
agent.reset()

# Persist run metadata

In [79]:
from src.run.utils import parse_collect_log, flatten_dict

In [80]:
collect_info = parse_collect_log(collect_fp)

In [81]:
if ARGS.LOG_TO_MLFLOW:
    import mlflow

    logger.info("Logging [COLLECT] info to MLflow...")
    mlflow.log_params(collect_info)
    logger.info("Logging config to MLflow...")
    mlflow.log_params(flatten_dict(cfg.model_dump(), "cfg", sep='.'))
    logger.info(f"Logging Retrieval Synthetic Eval Results to MLflow...")
    retrieval_evaluator.log_to_mlflow(cfg)
    logger.info(f"Logging Response Eval Results to MLflow...")
    try:
        response_evaluator.log_to_mlflow(
            cfg,
            'synthetic',
            response_synthetic_mean_scores_df,
            response_synthetic_deep_eval_df
        )
        response_evaluator.log_to_mlflow(
            cfg,
            'curated',
            response_curated_mean_scores_df,
            response_curated_deep_eval_df
        )
    except:
        ...

2024-08-13 21:51:42.768 | INFO     | __main__:<module>:4 - Logging [COLLECT] info to MLflow...
2024-08-13 21:51:42.809 | INFO     | __main__:<module>:6 - Logging config to MLflow...
/home/dvquys/frostmourne/study/vietai-genai03/final-project/review-rec-bot/.venv/lib/python3.11/site-packages/pydantic/main.py:364: UserWarning: Pydantic serializer warnings:
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
2024-08-13 21:51:42.872 | INFO     | __main__:<module>:8 - Logging Retrieval Synthetic Eval Results to MLflow...
2024-08-13 21:51:43.205 | INFO     | __main

# Clean up

In [82]:
if ARGS.LOG_TO_MLFLOW:
    mlflow.end_run()

# Chatbot UI

# Archive